# S&P500

In [ ]:
# Important libraries
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import random
import sklearn
from sklearn.metrics import auc
from matplotlib.gridspec import SubplotSpec
import joblib
import gpflow

# KNN
from scipy.spatial import KDTree

# Cond Gaussian
from sklearn.mixture import GaussianMixture
from scipy.stats import norm
from scipy.stats import pearsonr

# Weighted
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import weightedcalcs as wc

# Gaussian Copula
from scipy import stats
from scipy.stats import norm, truncnorm

# BNN
import torchbnn as bnn
import torch.nn as nn
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
from sklearn.model_selection import ParameterGrid
import random

# DER
from lightning.pytorch import Trainer, seed_everything
from lightning import LightningDataModule
from torch_uncertainty.models.mlp import mlp
from torch_uncertainty.losses import DERLoss
from torch_uncertainty.routines import RegressionRoutine
from torch_uncertainty.layers.distributions import NormalInverseGammaLayer
from lightning.pytorch.callbacks import EarlyStopping
import contextlib

data_label = "snp500"
seed = 2023

# File paths
fp_notebooks_folder = "./"
fp_code_folder = "../"
fp_processed_folder = os.path.join(fp_code_folder, "../processed_data")
fp_downsampled_folder = os.path.join(fp_processed_folder, "downsampled")
fp_downsampled_dropna_file = os.path.join(fp_downsampled_folder, "dropna.csv")
fp_downsampled_scaler_file = os.path.join(fp_downsampled_folder, "scaler.pkl")

fp_project_checkpoints = os.path.join(fp_code_folder, "checkpoints", data_label)
fp_tuning = os.path.join(fp_project_checkpoints, "tuning")
fp_project_models = os.path.join(fp_project_checkpoints, "models")
fp_project_predictions = os.path.join(fp_project_checkpoints, "predictions")
fp_project_pi_predictions = os.path.join(fp_project_checkpoints, "pi_predictions")
fp_project_model_evaluations = os.path.join(fp_project_checkpoints, "model_evaluation")
fp_project_consolidated_results = os.path.join(fp_project_checkpoints, "consolidated_results")
fp_time_log = os.path.join(fp_project_consolidated_results, "runtime.log")

def create_folder(fp):
    if not os.path.exists(fp):
        os.makedirs(fp)
        return True
    else:
        False

def create_all_seed_folders(cur_seed):
    fp_checkpoint_folders = [fp_project_models, fp_tuning, fp_project_predictions, fp_project_model_evaluations, fp_project_pi_predictions]
    for fp_folder in fp_checkpoint_folders:
        fp = os.path.join(fp_folder, str(cur_seed))
        create_folder(fp)
    print(f"All folders created for seed = {cur_seed}!")

batch_size = 64

# Create all folders
create_all_seed_folders(seed)
create_folder(fp_project_consolidated_results)

# Check GPU is available
print(tf.config.list_physical_devices('GPU'))

# function to show df
def display_df(df):
    display(df.head())
    print("Shape:", df.shape)

## Load Data

In [ ]:
df = pd.read_csv(fp_downsampled_dropna_file, index_col=0)
df

In [ ]:
predictors = df.columns[:60].to_list()
print(predictors)

In [ ]:
pred_cols_1 = [col for col in df.columns if "PredMin1" in col]
pred_cols_2 = [col for col in df.columns if "PredMin2" in col]
pred_cols_3 = [col for col in df.columns if "PredMin3" in col]
print(pred_cols_1)
print(pred_cols_2)
print(pred_cols_3)

In [ ]:
# Make train, validation and test sets
def train_valid_test_split(df, pred_cols):
    df_train, df_valid, df_test = df[df["train"]], df[df["valid"]], df[df["test"]]
    num_pred_cols = len(pred_cols)
    
    # Plot distribution of pred_col for each set
    fig, axes = plt.subplots(num_pred_cols, 3, figsize=(10, 2*num_pred_cols))
    for i, col in enumerate(pred_cols):
        axes[i, 0].hist(df_train[col])
        axes[i, 0].set_xlabel("Train")
        axes[i, 0].set_ylabel(col.split("_")[0])
        axes[i, 1].hist(df_valid[col])
        axes[i, 1].set_xlabel("Valid")
        axes[i, 2].hist(df_test[col])
        axes[i, 2].set_xlabel("Test")
    
    plt.tight_layout()

    return df_train, df_valid, df_test

df_train_1, df_valid_1, df_test_1 = train_valid_test_split(df, pred_cols=pred_cols_1)

In [ ]:
df_train_2, df_valid_2, df_test_2 = train_valid_test_split(df, pred_cols=pred_cols_2)

In [ ]:
df_train_3, df_valid_3, df_test_3 = train_valid_test_split(df, pred_cols=pred_cols_3)

## Timer Function

In [ ]:
class Timer:
    def __init__(self, seed):
        self.seed = seed

    def restart_timer(self):
        self.description = None
        self.start_time = None
        self.end_time = None
        self.duration = None
        
    def start(self, description):
        self.restart_timer()
        self.description = description
        self.start_time = time.time()

    def end(self):
        self.end_time = time.time()
        self.duration = self.end_time-self.start_time
        print(f"{self.description} took {self.duration}s. ")
        self._log_time()

    def _log_time(self):
        with open(fp_time_log, "a+") as myfile:
            myfile.write(f"{self.seed},{self.description},{self.duration}\n")

## RUE

In [ ]:
def set_seed(seed):
    tf.config.experimental.enable_op_determinism()
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.keras.utils.set_random_seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

def display_history(history, show_acc=False):
    if show_acc:
        fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(6, 2.5))
        axes[0].plot(history.history['loss'])
        axes[0].plot(history.history['val_loss'])
        axes[0].set_title('Model Loss')
        axes[0].set_ylabel('Loss')
        axes[0].set_xlabel('Epoch')
        axes[0].legend(['Train', 'Val'], loc='upper left')
        axes[1].plot(history.history['accuracy'])
        axes[1].plot(history.history['val_accuracy'])
        axes[1].set_title('Model Accuracy')
        axes[1].set_ylabel('Accuracy')
        axes[1].set_xlabel('Epoch')
        axes[1].legend(['Train', 'Val'], loc='upper left')
        axes[2].plot(history.history['f1_score'])
        axes[2].plot(history.history['val_f1_score'])
        axes[2].set_title('Model F1 Score')
        axes[2].set_ylabel('F1 Score')
        axes[2].set_xlabel('Epoch')
        axes[2].legend(['Train', 'Val'], loc='upper left')
    else:
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(3, 2.5))
        axes.plot(history.history['loss'])
        axes.plot(history.history['val_loss'])
        axes.set_title('Model Loss')
        axes.set_ylabel('Loss')
        axes.set_xlabel('Epoch')
        axes.legend(['Train', 'Val'], loc='upper left')
    plt.tight_layout()
    plt.show()

from keras import regularizers

class AE_Regressor:
    def __init__(self, encoder_width, encoder_depth, decoder_width, decoder_depth, predictors, output_features):
        from keras.layers import Dense, Dropout
        self.predictors = predictors
        self.output_features = output_features
        self.num_predictors = len(self.predictors)
        self.num_outputs = len(self.output_features)
        self.encoder_width = encoder_width
        self.encoder_depth = encoder_depth
        self.decoder_width = decoder_width
        self.decoder_depth = decoder_depth
        
        # Instantiate model layers
        self.inputs = tf.keras.Input(shape=(self.num_predictors,))
        self.encoder = tf.keras.Sequential(list(
            Dense(self.encoder_width, "relu") for i in range(self.encoder_depth) # , kernel_regularizer="l2"
        ), name="encoder")
        self.decoder = tf.keras.Sequential([
            Dense(self.decoder_width, "relu" , kernel_regularizer="l2") for i in range(self.decoder_depth-1) # , kernel_regularizer="l2"
        ]+[Dense(self.num_predictors)], name="decoder")
        self.regressor = tf.keras.Sequential([
            Dropout(rate=0.2),
            Dense(self.num_outputs)
        ], name="regressor")
    
    # Smote is external
    def train_regressor(
        self, train_X, train_y, val_X, val_y, batch_size, max_epochs, verbose, patience):
        from tensorflow.keras.callbacks import EarlyStopping
        # Define regressor
        pred = self.encoder(self.inputs)
        pred = self.regressor(pred)
        self.predictor = tf.keras.Model(inputs=self.inputs, outputs=pred, name="regression_model")
        # Train predictor
        self.predictor.compile(
            loss="mse",
            optimizer=tf.keras.optimizers.Adam()
        )
        es = EarlyStopping(
            monitor='val_loss', mode='min', verbose=1, patience=patience, restore_best_weights=True)
        self.predictor_history = self.predictor.fit(
            train_X, train_y, 
            epochs=max_epochs, 
            validation_data=(val_X, val_y),
            verbose=verbose,
            batch_size=batch_size,
            callbacks=[es],
        )
        print("- Regressor Training History")
        display_history(self.predictor_history)
        best_index = np.argmin(self.predictor_history.history['val_loss'])
        return (
            self.predictor_history.history['val_loss'][best_index], 
            best_index
        )
        
    def train_decoder(
        self, train_X, val_X, batch_size, max_epochs, verbose, patience):
        from tensorflow.keras.callbacks import EarlyStopping
        # Define AE
        self.encoder.trainable=False # Freeze weights
        x = self.encoder(self.inputs)
        x = self.decoder(x)
        self.ae = tf.keras.Model(inputs=self.inputs, outputs=x, name="ae_model")
        # Train AE
        self.ae.compile(
            loss="mse",
            optimizer=tf.keras.optimizers.Adam()
        )
        es = EarlyStopping(
            monitor='val_loss', mode='min', verbose=1, patience=patience, restore_best_weights=True)
        self.ae_history = self.ae.fit(
            train_X, train_X, 
            epochs=max_epochs, 
            validation_data=(val_X, val_X),
            verbose=verbose,
            batch_size=batch_size,
            callbacks=[es]
        )
        print("- Autoencoder Training History")
        display_history(self.ae_history)
        best_epoch = np.argmin(self.ae_history.history['val_loss'])
        return self.ae_history.history['val_loss'][best_epoch], best_epoch

    def replace_encoder_predictor(self, prev_model):
        self.encoder = prev_model.encoder
        self.regressor = prev_model.regressor
        self.encoder_width = prev_model.encoder_width
        self.encoder_depth = prev_model.encoder_depth

    def predict(self, inputs, with_mae=True, weighted=False, dropout_activated=False):
        # Encode
        encoder_output = self.encoder(inputs)
        
        # Get forecast result
        regressor_output = self.regressor(encoder_output, training=dropout_activated)
        if with_mae:
            # Reconstruct
            decoder_output = self.decoder(encoder_output)
            return regressor_output, decoder_output
        else:
            return regressor_output

    def get_config(self):
        return dict(
            encoder_width=self.encoder_width, 
            encoder_depth=self.encoder_depth, 
            decoder_width=self.decoder_width, 
            decoder_depth=self.decoder_depth, 
            predictors=self.predictors, 
            output_features=self.output_features
        )
        
def save_model(model, name, fp_checkpoints, override=False):
    import pickle
    model_folder = os.path.join(fp_checkpoints, name)
    if os.path.exists(model_folder):
        print("Model checkpoint already exists!")
        if not override:
            return
    else:
        os.makedirs(model_folder)
    
    # Save Parameters
    parameters_to_save = model.get_config()
    parameter_filename = os.path.join(fp_checkpoints, name, "parameters.pickle")
    with open(parameter_filename, 'wb') as handle:
        pickle.dump(parameters_to_save, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    # Save Model
    inputs = model.inputs
    encoder_output = model.encoder(inputs)
    decoder_output = model.decoder(encoder_output)
    regressor_output = model.regressor(encoder_output)
    model = tf.keras.Model(inputs, [regressor_output, decoder_output])
    model_filename = os.path.join(fp_checkpoints, name, "model.h5")
    model.save(model_filename)
    print("Model saved!")

def load_model(name, fp_checkpoints): 
    import pickle
    model_folder = os.path.join(fp_checkpoints, name)
    parameter_filename = os.path.join(fp_checkpoints, name, "parameters.pickle")
    model_filename = os.path.join(fp_checkpoints, name, "model.h5")
    
    if not os.path.exists(model_folder):
        print("model checkpoint does not exist!")
        return
    model = tf.keras.models.load_model(model_filename)
    with open(parameter_filename, 'rb') as handle:
        parameters = pickle.load(handle)
    
    ae_regressor = AE_Regressor(**parameters)
    ae_regressor.encoder = model.get_layer("encoder")
    ae_regressor.decoder = model.get_layer("decoder")
    ae_regressor.regressor = model.get_layer("regressor")
    
    return ae_regressor

def model_tuning_regressor(
    param_grid, predictors, pred_cols, train_df, valid_df, seed,
    batch_size=128, max_epochs=5000, verbose=1, patience=10):
    train_X, train_y = (
        train_df[predictors].values.astype('float32'), train_df[pred_cols].values.astype('float32'))
    valid_X, valid_y = (
        valid_df[predictors].values.astype('float32'), valid_df[pred_cols].values.astype('float32'))
    loss_list = []
    epoch_list = []
    time_spent_list = []
    tuning_df_list = []
    parameter_list = list(ParameterGrid(param_grid))
    pbar = tqdm(parameter_list)
    for param_dict in pbar:
        pbar.set_description(f"{param_dict}")
        set_seed(seed)
        regressor = AE_Regressor(**param_dict, predictors=predictors, output_features=pred_cols)
        start = time.time()
        val_loss, best_epoch = regressor.train_regressor(
            train_X, train_y, valid_X, valid_y, batch_size, max_epochs, verbose, patience)
        cur_param_dict = param_dict.copy()
        cur_param_dict["loss"] = val_loss
        cur_param_dict["epochs"] = best_epoch
        cur_param_dict["time/s"] = time.time()-start
        tuning_df_list.append(cur_param_dict)
    tuning_df = pd.DataFrame(tuning_df_list)
    best_param_idx = tuning_df['loss'].idxmin()
    tuning_df["best_hyperparameter"] = [True if i==best_param_idx else False for i in range(len(tuning_df))]
    best_param = parameter_list[best_param_idx]
    return tuning_df, best_param

def get_model_error_corr(predictor, x, y):
    y_pred, x_pred = predictor.predict(x)
    rue = np.mean(np.abs(x-x_pred), axis=-1)
    mae = np.mean(np.abs(y-y_pred), axis=-1)
    corr, pvalue = pearsonr(mae, rue)
    return corr

def model_tuning_decoder(
    param_grid, predictors, pred_cols, train_df, valid_df, seed, prev_model,
    atch_size=128, max_epochs=5000, verbose=1, patience=10):
    train_X, train_y = (
        train_df[predictors].values.astype('float32'), train_df[pred_cols].values.astype('float32'))
    valid_X, valid_y = (
        valid_df[predictors].values.astype('float32'), valid_df[pred_cols].values.astype('float32'))
    tuning_df_list = []
    parameter_list = list(ParameterGrid(param_grid))
    best_corr = -np.inf
    pbar = tqdm(parameter_list)
    for param_dict in pbar:
        pbar.set_description(f"{param_dict}, best corr: {best_corr:.5f}")
        set_seed(seed)
        predictor = AE_Regressor(**param_dict, predictors=predictors, output_features=pred_cols)
        predictor.replace_encoder_predictor(prev_model)
        start = time.time()
        val_loss, best_epoch = predictor.train_decoder(
            train_X, valid_X, batch_size, max_epochs, verbose, patience)
        timing = time.time()-start
        corr = get_model_error_corr(predictor, valid_X, valid_y)
        cur_param_dict = param_dict.copy()
        cur_param_dict["loss"] = val_loss
        cur_param_dict["corr"] = corr
        cur_param_dict["epochs"] = best_epoch
        cur_param_dict["time/s"] = timing
        tuning_df_list.append(cur_param_dict)
        best_corr = max(corr, best_corr)
    tuning_df = pd.DataFrame(tuning_df_list)
    best_param_idx = tuning_df["corr"].idxmax()
    tuning_df["best_hyperparameter"] = [True if i==best_param_idx else False for i in range(len(tuning_df))]
    best_param = parameter_list[best_param_idx]
    return tuning_df, best_param

def model_training(
    hp_dict, predictors, pred_cols, train_df, valid_df, seed,
    batch_size=128, max_epochs=5000, verbose=1, patience=10):
    set_seed(seed)
    timer = Timer(seed)
    
    # Get data
    train_X, train_y = (
        train_df[predictors].values.astype('float32'), train_df[pred_cols].values.astype('float32'))
    valid_X, valid_y = (
        valid_df[predictors].values.astype('float32'), valid_df[pred_cols].values.astype('float32'))
    
    # Train Regressor
    ae_regressor = AE_Regressor(**hp_dict, predictors=predictors, output_features=pred_cols)
    timer.start(description="Training Encoder and Predictor")
    valid_loss_regressor, best_epoch = ae_regressor.train_regressor(
        train_X, train_y, valid_X, valid_y, 
        batch_size, max_epochs, verbose, patience)
    timer.end()
    
    # Train decoder
    timer.start(description="Training Decoder")
    valid_loss_ae = ae_regressor.train_decoder(
        train_X, valid_X, batch_size, max_epochs, verbose, patience)
    timer.end()
    return ae_regressor

def model_training_predictor(
    hp_dict, predictors, pred_cols, train_df, valid_df, seed,
    batch_size=128, max_epochs=5000, verbose=1, patience=10):
    
    from tqdm import tqdm
    import time
    set_seed(seed)
    
    # Get data
    train_X, train_y = (
        train_df[predictors].values.astype('float32'), train_df[pred_cols].values.astype('float32'))
    valid_X, valid_y = (
        valid_df[predictors].values.astype('float32'), valid_df[pred_cols].values.astype('float32'))
    
    # Train Regressor
    ae_regressor = AE_Regressor(**hp_dict, predictors=predictors, output_features=pred_cols)
    
    start = time.time()
    # Train classifier
    valid_loss_regressor, best_epoch = ae_regressor.train_regressor(
        train_X, train_y, valid_X, valid_y, 
        batch_size, max_epochs, verbose, patience)
    
    # Train decoder
    # valid_loss_ae = ae_regressor.train_decoder(
    #     train_X, valid_X, batch_size, max_epochs, verbose, patience)
    
    print(f"Training took {time.time()-start}s.")
    return ae_regressor

def model_training_decoder(
    hp_dict, predictors, pred_cols, train_df, valid_df, seed, prev_model, 
    batch_size=128, max_epochs=5000, verbose=1, patience=10):
    
    from tqdm import tqdm
    import time
    set_seed(seed)
    
    # Get data
    train_X, train_y = (
        train_df[predictors].values.astype('float32'), train_df[pred_cols].values.astype('float32'))
    valid_X, valid_y = (
        valid_df[predictors].values.astype('float32'), valid_df[pred_cols].values.astype('float32'))
    
    # Train Regressor
    ae_regressor = AE_Regressor(**hp_dict, predictors=predictors, output_features=pred_cols)
    ae_regressor.replace_encoder_predictor(prev_model)
    
    start = time.time()
    # Train classifier
    # valid_loss_regressor, best_epoch = ae_regressor.train_regressor(
    #     train_X, train_y, valid_X, valid_y, 
    #     batch_size, max_epochs, verbose, patience)
    
    # Train decoder
    valid_loss_ae = ae_regressor.train_decoder(
        train_X, valid_X, batch_size, max_epochs, verbose, patience)
    
    print(f"Training took {time.time()-start}s.")
    return ae_regressor

def model_test_predictions(
    ae_regressor, df_train, df_test, pred_cols, predictors, regressor_label, pred_min, T=10, seed=seed):
    df_test = df_test.copy()
    set_seed(seed)
    timer = Timer(seed)
    test_X, test_y = (
        df_test[predictors].values.astype('float32'), df_test[pred_cols].values.astype('float32'))
    
    # MAE
    timer.start(description="Predicting with Decoder + Predictor")
    test_y_pred, test_X_reconstruction = ae_regressor.predict(inputs=test_X)
    timer.end()
    # - Add to df
    predicted_colnames = [col+"_direct"+regressor_label for col in pred_cols]
    reconstruction_colnames = [col+"_reconstruction"+regressor_label for col in predictors]
    rue_colname = "rue"
    df_test[predicted_colnames] = test_y_pred
    df_test[reconstruction_colnames] = test_X_reconstruction
    df_test[rue_colname] = np.mean(np.abs(test_X_reconstruction-test_X), axis=1)
    
    # For MC Dropout
    # - Sample with dropout
    timer.start(description="Predicting with MC Dropout")
    test_y_sample_preds = []
    for i in range(T):
        test_y_pred = ae_regressor.predict(inputs=test_X, dropout_activated=True, with_mae=False)
        test_y_sample_preds.append(test_y_pred)
    timer.end()
    test_y_sample_preds = np.array(test_y_sample_preds)
    test_y_sample_preds =  test_y_sample_preds.transpose((1, 2, 0))
    # - Get mean and std of all sample predictions
    test_y_mean_pred = np.mean(test_y_sample_preds, axis=-1)
    test_y_std_pred = np.std(test_y_sample_preds, axis=-1, ddof=1)

    # - Add to df
    predicted_mean_colnames = [col+"_mean"+regressor_label for col in pred_cols]
    predicted_std_colnames = [col+"_std"+regressor_label for col in pred_cols]
    mcd_colname = "mcd"
    df_test[predicted_mean_colnames] = test_y_mean_pred
    df_test[predicted_std_colnames] = test_y_std_pred
    df_test[mcd_colname] = np.mean(test_y_std_pred, axis=-1)
    
    if df_test['target_index'].dtype != "int64":
        df_test['target_index'] = df_test['target_index'].apply(lambda x: eval(x)[pred_min-1])
    
    return df_test

### Tune and Train Predictor

#### Tune Predictor

In [ ]:
rue_dict = {
    "t+1": {"train_df": df_train_1, "valid_df": df_valid_1, "test_df": df_test_1, "outputs": pred_cols_1},
    "t+2": {"train_df": df_train_2, "valid_df": df_valid_2, "test_df": df_test_2, "outputs": pred_cols_2},
    "t+3": {"train_df": df_train_3, "valid_df": df_valid_3, "test_df": df_test_3, "outputs": pred_cols_3},
}
# all_rue_best_hp = {}
for time_label, time_info_dict in tqdm(rue_dict.items(), total=len(rue_dict)):
    rue_tuning_df, rue_best_hp = model_tuning_regressor(
        param_grid=dict(
            encoder_width = [256, 512, 1028], # , 256, 512
            encoder_depth = [1, 2, 3], #  3, 4
            decoder_width = [128],
            decoder_depth = [2]
        ), predictors=predictors, pred_cols=time_info_dict["outputs"], 
        train_df=time_info_dict["train_df"], valid_df=time_info_dict["valid_df"], seed=seed,
        batch_size=batch_size, max_epochs=10000, verbose=1, patience=20
    )
    display(rue_tuning_df)
    rue_tuning_df.to_csv(os.path.join(fp_tuning, str(seed), f"tuning_rue_{time_label}.csv"))
    all_rue_best_hp[time_label] = rue_best_hp
joblib.dump(all_rue_best_hp, os.path.join(fp_tuning, str(seed), "all_rue_predictor_best_hp.joblib"))

In [ ]:
all_rue_predictor_best_hp = joblib.load(os.path.join(fp_tuning, str(seed), "all_rue_predictor_best_hp.joblib"))

#### Train Predictor

In [ ]:
rue_dict = {
    "t+1": {"train_df": df_train_1, "valid_df": df_valid_1, "test_df": df_test_1, "outputs": pred_cols_1},
    "t+2": {"train_df": df_train_2, "valid_df": df_valid_2, "test_df": df_test_2, "outputs": pred_cols_2},
    "t+3": {"train_df": df_train_3, "valid_df": df_valid_3, "test_df": df_test_3, "outputs": pred_cols_3},
}
all_rue_decoder_best_hp = {}
for time_label, time_info_dict in tqdm(rue_dict.items(), total=len(rue_dict)):
    best_predictor_hp = all_rue_predictor_best_hp[time_label]
    ae_regressor = model_training_predictor(
        best_predictor_hp, predictors=predictors, pred_cols=time_info_dict["outputs"], 
        train_df=time_info_dict["train_df"], valid_df = time_info_dict["valid_df"], seed=seed,
        batch_size=batch_size, max_epochs=10000, verbose=1, patience=20
    ) 
    save_model(model=ae_regressor, name=f"rue_predictor_{time_label}", fp_checkpoints=os.path.join(fp_project_models, str(seed)), override=True)

### Tune and Train Decoder

#### Tune Decoder

In [ ]:
rue_dict = {
    "t+1": {"train_df": df_train_1, "valid_df": df_valid_1, "test_df": df_test_1, "outputs": pred_cols_1},
    "t+2": {"train_df": df_train_2, "valid_df": df_valid_2, "test_df": df_test_2, "outputs": pred_cols_2},
    "t+3": {"train_df": df_train_3, "valid_df": df_valid_3, "test_df": df_test_3, "outputs": pred_cols_3},
}
all_rue_decoder_best_hp = {}
for time_label, time_info_dict in tqdm(rue_dict.items(), total=len(rue_dict)):
    best_predictor_hp = all_rue_predictor_best_hp[time_label]
    prev_model = load_model(name=f"rue_predictor_{time_label}", fp_checkpoints=os.path.join(fp_project_models, str(seed)))
    rue_tuning_df, rue_best_hp = model_tuning_decoder(
        param_grid=dict(
            encoder_width = [best_predictor_hp["encoder_width"]], # , 256, 512
            encoder_depth = [best_predictor_hp["encoder_depth"]], #  3, 4
            decoder_width = [256, 512, 1028], 
            decoder_depth = [1, 2, 3]
        ), predictors=predictors, pred_cols=time_info_dict["outputs"], 
        train_df=time_info_dict["train_df"], valid_df=time_info_dict["valid_df"], seed=seed,
        max_epochs=10000, verbose=1, patience=20, prev_model=prev_model
    )
    display(rue_tuning_df)
    rue_tuning_df.to_csv(os.path.join(fp_tuning, str(seed), f"tuning_rue_decoder_{time_label}.csv"))
    all_rue_decoder_best_hp[time_label] = rue_best_hp
joblib.dump(all_rue_decoder_best_hp, os.path.join(fp_tuning, str(seed), "all_rue_decoder_best_hp.joblib"))

In [ ]:
# With Regularisation (L2=0.01) and MSE: 0.33679
# With Regularisation and MAE: 0.296979
# Without Regularisation and MSE: 0.274257
# With Regularisation (L1) and MSE: 0.317686	
# With Regularisation (L2=) and MSE: 0.312463	

In [ ]:
all_rue_decoder_best_hp = joblib.load(os.path.join(fp_tuning, str(seed), "all_rue_decoder_best_hp.joblib"))
all_rue_decoder_best_hp

#### Train Decoder

In [ ]:
rue_dict = {
    "t+1": {"train_df": df_train_1, "valid_df": df_valid_1, "test_df": df_test_1, "outputs": pred_cols_1},
    "t+2": {"train_df": df_train_2, "valid_df": df_valid_2, "test_df": df_test_2, "outputs": pred_cols_2},
    "t+3": {"train_df": df_train_3, "valid_df": df_valid_3, "test_df": df_test_3, "outputs": pred_cols_3},
}
for time_label, time_info_dict in tqdm(rue_dict.items(), total=len(rue_dict)):
    prev_model = load_model(name=f"rue_predictor_{time_label}", fp_checkpoints=os.path.join(fp_project_models, str(seed)))
    hp_dict = all_rue_decoder_best_hp[time_label]
    ae_regressor = model_training_decoder(
        hp_dict, predictors=predictors, pred_cols=time_info_dict["outputs"], 
        train_df=time_info_dict["train_df"], valid_df = time_info_dict["valid_df"], seed=seed, prev_model=prev_model,
        batch_size=batch_size, max_epochs=10000, verbose=1, patience=20
    ) 
    save_model(model=ae_regressor, name=f"rue_{time_label}", fp_checkpoints=os.path.join(fp_project_models, str(seed)), override=True)

### Prediction

In [ ]:
rue_dict = {
    "t+1": {"train_df": df_train_1, "valid_df": df_valid_1, "test_df": df_test_1, "outputs": pred_cols_1},
    "t+2": {"train_df": df_train_2, "valid_df": df_valid_2, "test_df": df_test_2, "outputs": pred_cols_2},
    "t+3": {"train_df": df_train_3, "valid_df": df_valid_3, "test_df": df_test_3, "outputs": pred_cols_3},
}
for time_label, time_info_dict in tqdm(rue_dict.items(), total=len(rue_dict)):
    ae_regressor = load_model(name=f"rue_{time_label}", fp_checkpoints=os.path.join(fp_project_models, str(seed)))
    rue_valid_df = model_test_predictions(
        ae_regressor, df_train=time_info_dict["train_df"], df_test=time_info_dict["valid_df"], 
        pred_cols=time_info_dict["outputs"], predictors=predictors, regressor_label="_"+time_label, pred_min=int(time_label[-1]), T=10, seed=seed)
    rue_test_df = model_test_predictions(
        ae_regressor, df_train=time_info_dict["train_df"], df_test=time_info_dict["test_df"], 
        pred_cols=time_info_dict["outputs"], predictors=predictors, regressor_label="_"+time_label, pred_min=int(time_label[-1]), T=10, seed=seed)
    display(rue_test_df)
    rue_valid_df.to_csv(os.path.join(fp_project_predictions, str(seed), f"rue_valid_{time_label[-1]}.csv"))
    rue_test_df.to_csv(os.path.join(fp_project_predictions, str(seed), f"rue_test_{time_label[-1]}.csv"))

## GPR Model Training

In [ ]:
def model_training_gpr(
    predictors, pred_cols, train_df, valid_df, seed):

    timer = Timer(seed)
    
    # Get data
    train_X, train_y = (
        train_df[predictors].values.astype('float64'), train_df[pred_cols].values.astype('float64'))
    valid_X, valid_y = (
        valid_df[predictors].values.astype('float64'), valid_df[pred_cols].values.astype('float64'))

    rng = np.random.default_rng(seed)
    n_inducing = round(len(train_X)*0.001) # 0.1% of points as inducing points
    print("- Number of Inducing Points:", n_inducing)
    inducing_points = rng.choice(train_X, size=n_inducing, replace=False)

    start = time.time()
    
    def step_callback(step, variables, values):
        print(f"Step {step} {time.time()-start:.3f} s", end="\r")

    gpr = gpflow.models.SGPR(
        (train_X, train_y),
        kernel=gpflow.kernels.SquaredExponential(),
        inducing_variable=inducing_points,
    )
    opt = gpflow.optimizers.Scipy()
    print(f"Training started...")
    timer.start(description="Training GPR")
    opt.minimize(gpr.training_loss, gpr.trainable_variables, step_callback=step_callback)
    timer.end()
    # Train GPR
    # from sklearn.gaussian_process import GaussianProcessRegressor
    # from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
    # kernel = DotProduct() + WhiteKernel()
    # gpr = GaussianProcessRegressor(kernel=kernel, copy_X_train=False, random_state=seed)
    # print(f"Training started...")
    # start = time.time()
    # gpr.fit(train_X, train_y)
    
    
    return gpr

def model_test_predictions_gpr(
    gpr, df_test, pred_cols, predictors, regressor_label, pred_min, seed=seed):
    timer = Timer(seed)
    df_test = df_test.copy()
  
    test_X, test_y = (
        df_test[predictors].values.astype('float64'), df_test[pred_cols].values.astype('float64'))
    timer.start(description="Predicting with GPR")
    test_y_pred, test_std = gpr.compiled_predict_y(test_X)
    timer.end()

    # test_y_pred, test_std = gpr.predict(test_X, return_std=True)
#     print(test_std.shape)
    predicted_colnames = [col+"_gpr"+regressor_label for col in pred_cols]
    std_colnames = [col+"_gpr_std"+regressor_label for col in pred_cols]
    gpr_mean_std_colname = "gpr_std_mean"
    df_test[predicted_colnames] = test_y_pred
    df_test[std_colnames] = test_std
    df_test[gpr_mean_std_colname] = np.mean(test_std, axis=1)
    
    if df_test['target_index'].dtype != "int64":
        df_test['target_index'] = df_test['target_index'].apply(lambda x: eval(x)[pred_min-1])
    
    return df_test

def save_model_gpr(model, name, fp_checkpoints):
    model.compiled_predict_y = tf.function(
        lambda Xnew: model.predict_y(Xnew),
        input_signature=[tf.TensorSpec(shape=[None, len(predictors)], dtype=tf.float64)],
    )
    tf.saved_model.save(model, os.path.join(fp_checkpoints, name))
    print("Model saved!")

def load_model_gpr(name, fp_checkpoints): 
    import pickle
    model_folder = os.path.join(fp_checkpoints, name)
    
    if not os.path.exists(model_folder):
        print("model checkpoint does not exist!")
        return
    
    return tf.saved_model.load(model_folder)
    
def calculate_mse(df_test, pred_cols, model_label, regressor_label):
    y_true = df_test[pred_cols].values
    predicted_cols = [col+model_label+regressor_label for col in pred_cols]
    y_pred = df_test[predicted_cols].values
    return sklearn.metrics.mean_squared_error(y_true, y_pred)

### Predict T+1

In [ ]:
gpr_1 = model_training_gpr( # 88.14491534233093s.
    predictors=predictors, pred_cols=pred_cols_1, 
    train_df=df_train_1, valid_df = df_valid_1, seed=seed
) 

In [ ]:
save_model_gpr(model=gpr_1, name="gpr_1", fp_checkpoints=os.path.join(fp_project_models, str(seed)))

In [ ]:
gpr_1 = load_model_gpr(name="gpr_1", fp_checkpoints=os.path.join(fp_project_models, str(seed)))

In [ ]:
gpr_1_valid_pred = model_test_predictions_gpr(
    gpr=gpr_1, df_test=df_valid_1, pred_cols=pred_cols_1, 
    predictors=predictors, regressor_label="_t+1", pred_min=1)
gpr_1_valid_pred.to_csv(os.path.join(fp_project_predictions, str(seed), "gpr_valid_1.csv"))
gpr_1_test_pred = model_test_predictions_gpr(
    gpr=gpr_1, df_test=df_test_1, pred_cols=pred_cols_1, 
    predictors=predictors, regressor_label="_t+1", pred_min=1)
gpr_1_test_pred.to_csv(os.path.join(fp_project_predictions, str(seed), "gpr_test_1.csv"))

In [ ]:
calculate_mse(df_test=gpr_1_test_pred, pred_cols=pred_cols_1, model_label="_gpr", regressor_label="_t+1")

In [ ]:
# gpr_1 = load_model_gpr(name="gpr_1", fp_checkpoints=os.path.join(fp_project_models, str(seed)))

### Predict T+2

In [ ]:
gpr_2 = model_training_gpr( 
    predictors=predictors, pred_cols=pred_cols_2, 
    train_df=df_train_2, valid_df = df_valid_2, seed=seed
) 

In [ ]:
save_model_gpr(model=gpr_2, name="gpr_2", fp_checkpoints=os.path.join(fp_project_models, str(seed)))

In [ ]:
gpr_2 = load_model_gpr(name="gpr_2", fp_checkpoints=os.path.join(fp_project_models, str(seed)))

In [ ]:
gpr_2_valid_pred = model_test_predictions_gpr(
    gpr=gpr_2, df_test=df_valid_2, pred_cols=pred_cols_2, 
    predictors=predictors, regressor_label="_t+2", pred_min=2)
gpr_2_valid_pred.to_csv(os.path.join(fp_project_predictions, str(seed), "gpr_valid_2.csv"))
gpr_2_test_pred = model_test_predictions_gpr(
    gpr=gpr_2, df_test=df_test_2, pred_cols=pred_cols_2, 
    predictors=predictors, regressor_label="_t+2", pred_min=2)
gpr_2_test_pred.to_csv(os.path.join(fp_project_predictions, str(seed), "gpr_test_2.csv"))

In [ ]:
calculate_mse(df_test=gpr_2_test_pred, pred_cols=pred_cols_2, model_label="_gpr", regressor_label="_t+2")

In [ ]:
# gpr_2 = load_model_gpr(name="gpr_2", fp_checkpoints=os.path.join(fp_project_models, str(seed)))

### Predict T+3

In [ ]:
gpr_3 = model_training_gpr( 
    predictors=predictors, pred_cols=pred_cols_3, 
    train_df=df_train_3, valid_df = df_valid_3, seed=seed
) 

In [ ]:
save_model_gpr(model=gpr_3, name="gpr_3", fp_checkpoints=os.path.join(fp_project_models, str(seed)))

In [ ]:
gpr_3 = load_model_gpr(name="gpr_3", fp_checkpoints=os.path.join(fp_project_models, str(seed)))

In [ ]:
gpr_3_valid_pred = model_test_predictions_gpr(
    gpr=gpr_3, df_test=df_valid_3, pred_cols=pred_cols_3, 
    predictors=predictors, regressor_label="_t+3", pred_min=3)
gpr_3_valid_pred.to_csv(os.path.join(fp_project_predictions, str(seed), "gpr_valid_3.csv"))
gpr_3_test_pred = model_test_predictions_gpr(
    gpr=gpr_3, df_test=df_test_3, pred_cols=pred_cols_3, 
    predictors=predictors, regressor_label="_t+3", pred_min=3)
gpr_3_test_pred.to_csv(os.path.join(fp_project_predictions, str(seed), "gpr_test_3.csv"))

In [ ]:
calculate_mse(df_test=gpr_3_test_pred, pred_cols=pred_cols_3, model_label="_gpr", regressor_label="_t+3")

In [ ]:
# gpr_3 = load_model_gpr(name="gpr_3", fp_checkpoints=os.path.join(fp_project_models, str(seed)))

## InferNoise

In [ ]:
def mae_fn(y_true, y_pred):
    return np.mean(np.abs(y_true-y_pred), axis=-1)

def tune_infernoise(ae_predictor, stddev_list, valid_df, inputs, outputs, seed, T, regressor_label):
    corr_list = []
    loss_list = []
    for stddev in tqdm(stddev_list):
        valid_pred_df = infernoise_test_predictions(ae_predictor, valid_df, inputs, outputs, seed, T, stddev, regressor_label, False)
        loss = valid_pred_df["infernoise_mae"]
        ue = valid_pred_df["infernoise_uncertainty"]
        corr, _ = pearsonr(ue, loss)
        loss_list.append(valid_pred_df["infernoise_mae"].mean())
        corr_list.append(corr)
    tuning_df = pd.DataFrame({"std": stddev_list, "correlation": corr_list, "loss":loss_list})
    tuning_df["best_hyperparameter"] = False
    best_index = tuning_df["loss"].idxmin()
    tuning_df.iloc[best_index, -1] = True
    return tuning_df

def infernoise_test_predictions(ae_predictor, test_df, inputs, outputs, seed, T, stddev, regressor_label, time_run=True):
    timer = Timer(seed=seed)
    test_df = test_df.copy()
    # Process data
    test_X, test_y = (test_df[inputs].values.astype('float32'), test_df[outputs].values.astype('float32'))
    
    set_seed(seed)

    # Define model
    input_layer = tf.keras.Input(shape=(len(inputs),))
    gaussian_noise_layer = tf.keras.layers.GaussianNoise(stddev, seed=seed)
    x = ae_predictor.encoder(input_layer)
    x = gaussian_noise_layer(x)
    x = ae_predictor.regressor.layers[-1](x) # To ignore dropout layer
    infernoise_model = tf.keras.Model(inputs=input_layer, outputs=x, name="infernoise_model")
    
    # For Infer Noise
    # - Sample with infer noise
    test_y_sample_preds = [] # T, num samples, output classes
    if time_run:
        timer.start(description="Predicting with Infer-Noise")
    for i in range(T):
        test_y_pred = infernoise_model(test_X, training=True)
        # print(test_y_pred[:5])
        test_y_sample_preds.append(test_y_pred)
    if time_run:
        timer.end()
    
    test_y_sample_preds = np.array(test_y_sample_preds)
    test_y_sample_preds =  test_y_sample_preds.transpose((1, 2, 0)) # num samples, predicted features, T
    
    # - Get mean prediction 
    # num samples, predicted features
    test_y_mean_pred = np.mean(test_y_sample_preds, axis=-1)
    predicted_infernoise_colnames = [col + "_infernoise"+regressor_label for col in outputs]
    test_df[predicted_infernoise_colnames] = test_y_mean_pred
    
    # - Get loss for each output
    test_y_mc_loss = mae_fn(y_true=test_y, y_pred=test_y_mean_pred)
    test_df["infernoise_mae"] = test_y_mc_loss
    
    # - Uncertainty score (for regression)
    #   - Calculate std of predictions
    test_y_sd_pred = np.mean(np.std(test_y_sample_preds, axis=-1, ddof=1), axis=-1)
    test_df["infernoise_uncertainty"] = test_y_sd_pred

    return test_df

def display_tuning_df(tuning_df):
    def highlight_sentiment(row):
        if row["best_hyperparameter"]:
            return ['background-color: #cafae6'] * len(row)
        else:
            return [''] * len(row)
    return tuning_df.style.apply(highlight_sentiment, axis=1)


### Tuning

In [ ]:
infernoise_dict = {
    "t+1": {"model_name": "rue_t+1", "valid_df": df_valid_1, "test_df": df_test_1, "outputs": pred_cols_1},
    "t+2": {"model_name": "rue_t+2", "valid_df": df_valid_2, "test_df": df_test_2, "outputs": pred_cols_2},
    "t+3": {"model_name": "rue_t+3", "valid_df": df_valid_3, "test_df": df_test_3, "outputs": pred_cols_3},
}
infernoise_hp_dict = {}
for time_label, info_dict in infernoise_dict.items():
    print(f"{time_label}:")
    ae_regressor = load_model(name=info_dict["model_name"], fp_checkpoints=os.path.join(fp_project_models, str(seed)))
    infernoise_tuning_df = tune_infernoise(
        ae_regressor, stddev_list=[0.00001, 0.00005, 0.0001, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5], 
        valid_df=info_dict["valid_df"], inputs=predictors, outputs=info_dict["outputs"], seed=seed, T=10, regressor_label="_"+time_label
    )
    display(display_tuning_df(infernoise_tuning_df))
    infernoise_hp_dict[time_label] = infernoise_tuning_df.iloc[infernoise_tuning_df["loss"].argmin(), 0]

In [ ]:
joblib.dump(infernoise_hp_dict, os.path.join(fp_tuning, "all_infernoise_best_hp.joblib"))

In [ ]:
infernoise_hp_dict = joblib.load(os.path.join(fp_tuning, "all_infernoise_best_hp.joblib"))
infernoise_hp_dict

### Prediction

In [ ]:
for time_label, info_dict in infernoise_dict.items():
    print(f"{time_label}:")
    ae_regressor = load_model(name=info_dict["model_name"], fp_checkpoints=os.path.join(fp_project_models, str(seed)))
    infernoise_valid_df = infernoise_test_predictions(
         ae_regressor, test_df=info_dict["valid_df"], inputs=predictors, outputs=info_dict["outputs"], regressor_label="_"+time_label, 
        seed=seed, T=10, stddev=infernoise_hp_dict[time_label])
    infernoise_test_df = infernoise_test_predictions(
         ae_regressor, test_df=info_dict["test_df"], inputs=predictors, outputs=info_dict["outputs"], regressor_label="_"+time_label, 
        seed=seed, T=10, stddev=infernoise_hp_dict[time_label])
    display(infernoise_test_df)
    infernoise_valid_df.to_csv(os.path.join(fp_project_predictions, str(seed), f"infernoise_valid_{time_label[-1]}.csv"))
    infernoise_test_df.to_csv(os.path.join(fp_project_predictions, str(seed), f"infernoise_test_{time_label[-1]}.csv"))

## BNN

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def mae_fn(y_true, y_pred):
    return np.mean(np.abs(y_true-y_pred), axis=-1)

def set_seed_pytorch(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

class TabularDataset(Dataset):
    def __init__(self, df, feat_cols, target_cols):
        self.data_df = df
        self.feat_cols = feat_cols
        self.target_cols = target_cols

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        row = self.data_df.iloc[idx]
        features = torch.from_numpy(row[self.feat_cols].values.astype(float)).float()
        label = torch.from_numpy(row[self.target_cols].values.astype(float)).float()
        return features, label

def instantiate_bnn_model(num_layers, width, num_inputs, num_outputs, seed):
    set_seed_pytorch(seed)
    layers = []
    in_features, out_features = num_inputs, width
    for i in range(num_layers):
        if (i == (num_layers-1)):
            out_features = num_outputs
        layers.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=in_features, out_features=out_features))
        if (i != num_layers-1):
            layers.append(nn.ReLU())
        in_features = width
    return nn.Sequential(*layers)

def train_bnn_model(bnn_model, train_df, valid_df, feat_cols, target_cols, epochs, patience, seed, fp_model):
    set_seed_pytorch(seed)
    # Prepare dataset
    train_ds = TabularDataset(df=train_df, feat_cols=feat_cols, target_cols=target_cols)
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

    # Prepare dataset
    set_seed_pytorch(seed)
    valid_ds = TabularDataset(df=valid_df, feat_cols=feat_cols, target_cols=target_cols)
    valid_dl = DataLoader(valid_ds, batch_size=batch_size, shuffle=False)
    
    mse_loss = nn.MSELoss()
    kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
    kl_weight = 0.1
    optimizer = optim.Adam(bnn_model.parameters(), lr=0.001)

    best_epoch, best_val_loss, patience_count = -1, np.inf, 0

    bnn_model.to(device)

    with tqdm(range(epochs), total=epochs) as pbar:
        for epoch in pbar:
            for x_batch, y_batch in train_dl:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                pred = bnn_model(x_batch)
                mse = mse_loss(pred, y_batch)
                kl = kl_loss(bnn_model)
                cost = mse + kl_weight*kl
                
                optimizer.zero_grad()
                cost.backward()
                optimizer.step()
                
                x_batch = x_batch.detach().cpu()
                y_batch = y_batch.detach().cpu()
                mse = mse.detach().cpu()
                kl = kl.detach().cpu()
                cost = cost.detach().cpu()
                
            # Evaluate performance on validation set
            valid_pred = predict_bnn_model(bnn_model, valid_dl, feat_cols, target_cols, seed=seed, silent=True)
            valid_loss = evaluate_bnn_perf(valid_df, feat_cols, target_cols, valid_pred)
            pbar.set_description(f"valid_loss: {valid_loss:.5f}")

            # Early stopping
            if valid_loss < best_val_loss:
                best_epoch, best_val_loss = epoch, valid_loss
                patience_count = 0
                torch.save(bnn_model, fp_model)
            else:
                patience_count += 1
                if patience_count > patience:
                    print(f"Early stopping! Model achieved best performance at Epoch {best_epoch} with loss = {best_val_loss}.")
                    break
    
    return best_val_loss, best_epoch
        
def predict_bnn_model(bnn_model, dl, feat_cols, target_cols, seed, silent):
    set_seed_pytorch(seed)
    # Send model to gpu
    bnn_model.to(device)
    # Predictions
    all_pred = []
    # Predict with model
    with tqdm(dl, total=len(dl), disable=silent) as pbar:
        for x_batch, y_batch in pbar:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            pred = bnn_model(x_batch)
            all_pred.append(pred.detach().cpu())
    return torch.cat(all_pred)

def evaluate_bnn_perf(df, feat_cols, target_cols, pred):
    y = df[target_cols].values
    mse = torch.mean(torch.square(pred-y)).item()
    return mse

def evaluate_bnn(bnn_model, df, feat_cols, target_cols, seed):
    ds = TabularDataset(df=df, feat_cols=feat_cols, target_cols=target_cols)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False)
    pred = predict_bnn_model(bnn_model, dl, feat_cols, target_cols, seed=seed, silent=True)
    return evaluate_bnn_perf(df, feat_cols, target_cols, pred)

def tune_bnn_model(param_grid, train_df, valid_df, feat_cols, target_cols, epochs, patience, seed, fp_model):
    parameter_list = list(ParameterGrid(param_grid))
    loss_list, time_list, epoch_list = [], [], []
    with tqdm(parameter_list) as pbar:
        for param_dict in pbar:
            bnn_model = instantiate_bnn_model(seed=seed, num_inputs=len(feat_cols), num_outputs=len(target_cols), **param_dict)
            start = time.time()
            loss, best_epoch = train_bnn_model(bnn_model, train_df, valid_df, feat_cols, target_cols, epochs, patience, seed, fp_model)
            time_list.append(time.time()-start)
            epoch_list.append(best_epoch)
            loss_list.append(loss)
    tuning_df = pd.DataFrame(parameter_list)
    tuning_df["loss"] = loss_list
    tuning_df["epoch"] = epoch_list
    tuning_df["time/s"] = time_list
    best_index = np.argmin(tuning_df["loss"])
    tuning_df["best_hyperparameter"] = False
    tuning_df.iloc[best_index, -1] = True
    return tuning_df, parameter_list[best_index]

def train_model_w_best_param(best_param, train_df, valid_df, feat_cols, target_cols, epochs, patience, seed, fp_model):
    bnn_model =  instantiate_bnn_model(seed=seed, num_inputs=len(feat_cols), num_outputs=len(target_cols), **best_param)
    timer = Timer(seed=seed)
    timer.start(description="Training BNN")
    train_bnn_model(bnn_model, train_df, valid_df, feat_cols, target_cols, epochs, patience, seed, fp_model)
    timer.end()
    return torch.load(fp_model)

def bnn_model_prediction(bnn_model, test_df, feat_cols, target_cols, T, seed,  regressor_label):
    test_df = test_df.copy()
    set_seed_pytorch(seed)
    # Prepare dataset
    test_ds = TabularDataset(df=test_df, feat_cols=feat_cols, target_cols=target_cols)
    test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=False)
    seed_list = list(range(seed, seed+T))
    all_logits = []
    timer = Timer(seed=seed)
    timer.start(description="Predicting with BNN")
    for cur_seed in tqdm(seed_list):
        logits = predict_bnn_model(bnn_model, test_dl, feat_cols, target_cols, seed=cur_seed, silent=True)
        all_logits.append(logits)
    timer.end()
    
    all_logits = torch.stack(all_logits) # T, N, O
    test_y_pred = all_logits.mean(axis=0) # N, O
    test_y_std = all_logits.std(axis=0) # N, O
    test_y = torch.from_numpy(test_df[target_cols].values).float()
    
    test_df["bnn_uncertainty"] = test_y_std.mean(axis=-1) # N
    test_df["bnn_mae"] = mae_fn(y_true=test_y.numpy(), y_pred=test_y_pred.numpy())
    predicted_colnames = [col + "_bnn_"+ regressor_label for col in target_cols]
    test_df[predicted_colnames] = test_y_pred
    return test_df

### Tuning

In [ ]:
best_width_1=256
ae_regressor_1 = model_training(
    width=best_width_1, predictors=predictors, pred_cols=pred_cols_1, 
    train_df=df_train_1, valid_df = df_valid_1, seed=seed,
    batch_size=2048, max_epochs=10000, verbose=1, patience=20
) 

In [ ]:
bnn_dict = {
    "t+1": {"train_df": df_train_1, "valid_df": df_valid_1, "test_df": df_test_1, "outputs": pred_cols_1},
    "t+2": {"train_df": df_train_2, "valid_df": df_valid_2, "test_df": df_test_2, "outputs": pred_cols_2},
    "t+3": {"train_df": df_train_3, "valid_df": df_valid_3, "test_df": df_test_3, "outputs": pred_cols_3},
}
all_bnn_best_hp = {}
for time_label, time_info_dict in bnn_dict.items():
    fp_model = os.path.join(fp_project_models, str(seed), f"bnn_{time_label}.pt")
    bnn_tuning_df, bnn_best_hp = tune_bnn_model(
        param_grid={"num_layers":[2, 3], "width":[32, 64, 128, 256]}, 
        train_df=time_info_dict["train_df"], valid_df=time_info_dict["valid_df"], 
        feat_cols=predictors, target_cols=time_info_dict["outputs"], epochs=500, patience=5, seed=seed, fp_model=fp_model)
    display(bnn_tuning_df)
    bnn_tuning_df.to_csv(os.path.join(fp_tuning, str(seed), f"tuning_bnn_{time_label}.csv"))
    all_bnn_best_hp[time_label] = bnn_best_hp
joblib.dump(all_bnn_best_hp, os.path.join(fp_tuning, "all_bnn_best_hp.joblib"))

In [ ]:
all_bnn_best_hp = joblib.load(os.path.join(fp_tuning, str(seed), "all_bnn_best_hp.joblib"))
print(all_bnn_best_hp)

### Training

In [ ]:
bnn_dict = {
    "t+1": {"train_df": df_train_1, "valid_df": df_valid_1, "test_df": df_test_1, "outputs": pred_cols_1},
    "t+2": {"train_df": df_train_2, "valid_df": df_valid_2, "test_df": df_test_2, "outputs": pred_cols_2},
    "t+3": {"train_df": df_train_3, "valid_df": df_valid_3, "test_df": df_test_3, "outputs": pred_cols_3},
}
for time_label, time_info_dict in tqdm(bnn_dict.items(), total=len(bnn_dict)):
    fp_model = os.path.join(fp_project_models, str(seed), f"bnn_{time_label}.pt")
    bnn_best_hp = all_bnn_best_hp[time_label]
    bnn_model = train_model_w_best_param(
        best_param=bnn_best_hp, train_df=time_info_dict["train_df"], valid_df=time_info_dict["valid_df"], 
        feat_cols=predictors, target_cols=time_info_dict["outputs"],
        epochs=500, patience=5, seed=seed, fp_model=fp_model
    )

### Prediction

In [ ]:
bnn_dict = {
    "t+1": {"train_df": df_train_1, "valid_df": df_valid_1, "test_df": df_test_1, "outputs": pred_cols_1},
    "t+2": {"train_df": df_train_2, "valid_df": df_valid_2, "test_df": df_test_2, "outputs": pred_cols_2},
    "t+3": {"train_df": df_train_3, "valid_df": df_valid_3, "test_df": df_test_3, "outputs": pred_cols_3},
}
for time_label, time_info_dict in tqdm(bnn_dict.items(), total=len(bnn_dict)):
    fp_model = os.path.join(fp_project_models, str(seed), f"bnn_{time_label}.pt")
    bnn_model = torch.load(fp_model)
    bnn_valid_df = bnn_model_prediction(bnn_model, time_info_dict["valid_df"], feat_cols=predictors, target_cols=time_info_dict["outputs"], T=10, seed=seed, regressor_label=time_label)
    bnn_test_df = bnn_model_prediction(bnn_model, time_info_dict["test_df"], feat_cols=predictors, target_cols=time_info_dict["outputs"], T=10, seed=seed, regressor_label=time_label)
    display(bnn_test_df)
    bnn_valid_df.to_csv(os.path.join(fp_project_predictions, str(seed), f"bnn_valid_{time_label[-1]}.csv"))
    bnn_test_df.to_csv(os.path.join(fp_project_predictions, str(seed), f"bnn_test_{time_label[-1]}.csv"))

## DER

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

enable_print  = print
disable_print = lambda *x, **y: None

def mae_fn(y_true, y_pred):
    return np.mean(np.abs(y_true-y_pred), axis=-1)

class TabularDataset(Dataset):
    def __init__(self, df, feat_cols, target_cols):
        self.data_df = df
        self.feat_cols = feat_cols
        self.target_cols = target_cols

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        row = self.data_df.iloc[idx]
        features = torch.from_numpy(row[self.feat_cols].values.astype(float)).float()
        label = torch.from_numpy(row[self.target_cols].values.astype(float)).float()
        return features, label

def optim_regression(
    model: nn.Module, learning_rate: float = 0.001):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0,)
    return optimizer

def train_der_w_param(n_hidden_layers, hidden_width, train_df, valid_df, inputs, outputs, seed, max_epochs=500, patience=5, time_training=True):
    seed_everything(seed, workers=True)
    timer = Timer(seed=seed)
    # Data
    train_ds = TabularDataset(df=train_df, feat_cols=inputs, target_cols=outputs)
    valid_ds = TabularDataset(df=valid_df, feat_cols=inputs, target_cols=outputs)
    datamodule = LightningDataModule.from_datasets(train_ds, val_dataset=valid_ds, test_dataset=valid_ds, batch_size=batch_size, num_workers=63)
    datamodule.training_task = "regression"
    # print("hidden_dims:", [hidden_width for _ in range(n_hidden_layers)])
    # Model
    model = mlp(
        in_features=len(inputs),
        num_outputs=4*len(outputs),
        hidden_dims=[hidden_width for _ in range(n_hidden_layers)],
        final_layer=NormalInverseGammaLayer,
        final_layer_args={"dim": len(outputs)},
    )
    
    # Training
    loss = DERLoss(reg_weight=1e-2)
    routine = RegressionRoutine(
        probabilistic=True,
        output_dim=len(outputs),
        model=model,
        loss=loss,
        optim_recipe=optim_regression(model),
    )
    early_stopping = EarlyStopping('val/MSE', patience=patience, verbose=True, mode='min')

    trainer = Trainer(accelerator="gpu", devices=1, max_epochs=max_epochs, enable_progress_bar=True, callbacks=[early_stopping])
    with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
        if time_training:
            timer.start(description="Training DER")
        trainer.fit(model=routine, datamodule=datamodule)
        if time_training:
            timer.end()
        result = trainer.test(model=routine, datamodule=datamodule)

    return model, result[0]

def der_model_prediction(der_model, test_df, feat_cols, target_cols, seed, silent, regressor_label):
    seed_everything(seed, workers=True)
    test_df = test_df.copy()

    timer = Timer(seed=seed)
    timer.start(description="Predicting with DER")
    with torch.no_grad():
        x = torch.from_numpy(test_df[feat_cols].values).float()
        dists = der_model(x)
        means = dists.loc
        std = torch.sqrt(dists.variance_loc)
    timer.end()
    
    test_y = torch.from_numpy(test_df[target_cols].values).float()
    
    test_df["der_uncertainty"] = std.mean(axis=-1) # N
    test_df["der_mae"] = mae_fn(y_true=test_y.numpy(), y_pred=means.numpy())
    predicted_colnames = [col + "_der_"+regressor_label for col in target_cols]
    test_df[predicted_colnames] = means
    return test_df

def tune_der_model(param_grid, train_df, valid_df, feat_cols, target_cols, epochs, patience, seed,):
    parameter_list = list(ParameterGrid(param_grid))
    loss_list, time_list = [], []
    with tqdm(parameter_list) as pbar:
        for param_dict in pbar:
            start = time.time()
            der_model, result = train_der_w_param(
                train_df=train_df, valid_df=valid_df, 
                inputs=feat_cols, outputs=target_cols,
                seed=seed, max_epochs=epochs, patience=patience, **param_dict
            )
            # print(result)
            time_list.append(time.time()-start)
            loss_list.append(result['test/MSE'])
    tuning_df = pd.DataFrame(parameter_list)
    tuning_df["loss"] = loss_list
    tuning_df["time/s"] = time_list
    best_index = np.argmin(tuning_df["loss"])
    tuning_df["best_hyperparameter"] = False
    tuning_df.iloc[best_index, -1] = True
    return tuning_df, parameter_list[best_index]

### Tuning

In [ ]:
der_dict = {
    "t+1": {"train_df": df_train_1, "valid_df": df_valid_1, "test_df": df_test_1, "outputs": pred_cols_1},
    "t+2": {"train_df": df_train_2, "valid_df": df_valid_2, "test_df": df_test_2, "outputs": pred_cols_2},
    "t+3": {"train_df": df_train_3, "valid_df": df_valid_3, "test_df": df_test_3, "outputs": pred_cols_3},
}
all_der_best_hp = {}
for time_label, time_info_dict in tqdm(der_dict.items(), total=len(der_dict)):
    der_tuning_df, der_best_hp = tune_der_model(
        param_grid={
            "n_hidden_layers":[2, 3, 4],
            "hidden_width": [128, 256, 512]}, 
        train_df=time_info_dict["train_df"], valid_df=time_info_dict["valid_df"], 
        feat_cols=predictors, target_cols=time_info_dict["outputs"], epochs=500, patience=5, seed=seed
    )
    der_tuning_df.to_csv(os.path.join(fp_tuning, str(seed), f"tuning_der_{time_label}.csv"))
    all_der_best_hp[time_label] = der_best_hp
    display(der_tuning_df)
joblib.dump(all_der_best_hp, os.path.join(fp_tuning, str(seed), "all_der_best_hp.joblib"))

In [ ]:
all_der_best_hp = joblib.load(os.path.join(fp_tuning, "all_der_best_hp.joblib"))
all_der_best_hp

### Training

In [ ]:
der_dict = {
    "t+1": {"train_df": df_train_1, "valid_df": df_valid_1, "test_df": df_test_1, "outputs": pred_cols_1},
    "t+2": {"train_df": df_train_2, "valid_df": df_valid_2, "test_df": df_test_2, "outputs": pred_cols_2},
    "t+3": {"train_df": df_train_3, "valid_df": df_valid_3, "test_df": df_test_3, "outputs": pred_cols_3},
}
for time_label, time_info_dict in tqdm(der_dict.items(), total=len(der_dict)):
    fp_model = os.path.join(fp_project_models, str(seed), f"der_{time_label}.pt")
    der_model, _ = train_der_w_param(
        **all_der_best_hp[time_label], 
        train_df=time_info_dict["train_df"], valid_df=time_info_dict["valid_df"], 
        inputs=predictors, outputs=time_info_dict["outputs"],
        seed=seed, max_epochs=500, patience=5
    )
    torch.save(der_model, fp_model)

### Prediction

In [ ]:
der_dict = {
    "t+1": {"train_df": df_train_1, "valid_df": df_valid_1, "test_df": df_test_1, "outputs": pred_cols_1},
    "t+2": {"train_df": df_train_2, "valid_df": df_valid_2, "test_df": df_test_2, "outputs": pred_cols_2},
    "t+3": {"train_df": df_train_3, "valid_df": df_valid_3, "test_df": df_test_3, "outputs": pred_cols_3},
}
for time_label, time_info_dict in tqdm(der_dict.items(), total=len(der_dict)):
    fp_model = os.path.join(fp_project_models, str(seed), f"der_{time_label}.pt")
    der_model = torch.load(fp_model)
    der_valid_df = der_model_prediction(
        der_model, test_df=time_info_dict["valid_df"], feat_cols=predictors, target_cols=time_info_dict["outputs"], seed=seed, silent=False, regressor_label=time_label)
    der_test_df = der_model_prediction(
        der_model, test_df=time_info_dict["test_df"], feat_cols=predictors, target_cols=time_info_dict["outputs"], seed=seed, silent=False, regressor_label=time_label)
    display(der_test_df)
    der_valid_df.to_csv(os.path.join(fp_project_predictions, str(seed), f"der_valid_{time_label[-1]}.csv"))
    der_test_df.to_csv(os.path.join(fp_project_predictions, str(seed), f"der_test_{time_label[-1]}.csv"))

## Generate Prediction Interval

#### Load Predictions

In [ ]:
def load_scaler(fp_downsampled_scaler_file):
    import pickle 
    with open(fp_downsampled_scaler_file, 'rb') as handle:
        scaler = pickle.load(handle)
    return scaler

def load_all_predictions(time_label, split="test"):
    columns = []
    df_list = [] 
    for model in ["rue", "gpr", "infernoise", "bnn", "der"]: # Update this when you have a new ue
        fp = os.path.join(fp_project_predictions, str(seed), f"{model}_{split}_{time_label}.csv")
        df = pd.read_csv(fp, index_col=0)
        # find new columns not in current column list
        new_cols = [col for col in df.columns if col not in columns]
        columns.extend(new_cols)
        df_list.append(df[new_cols])
    return pd.concat(df_list, axis=1)

# Save all predictions
def save_df_dict(df_dict, seed):
    for time_label, time_info in tqdm(df_dict.items()):
        fp_seed_folder = os.path.join(fp_project_pi_predictions, str(seed))
        create_folder(fp_seed_folder)
        val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
        val_df.to_csv(os.path.join(fp_seed_folder, "val_"+time_label+".csv"))
        test_df.to_csv(os.path.join(fp_seed_folder, "test_"+time_label+".csv"))
        joblib.dump(pred_cols, os.path.join(fp_seed_folder, "pred_cols_"+time_label+".joblib"))
    print("Saved df_dict!")

# Load all predictions
def load_df_dict(seed, time_labels=["t+1", "t+2", "t+3"]):
    df_dict = {}
    for time_label in tqdm(time_labels):
        fp_seed_folder = os.path.join(fp_project_pi_predictions, str(seed))
        val_df = pd.read_csv(os.path.join(fp_seed_folder, "val_"+time_label+".csv"), index_col=0)
        val_df = val_df.loc[:, ~val_df.columns.str.contains('^Unnamed')]
        test_df = pd.read_csv(os.path.join(fp_seed_folder, "test_"+time_label+".csv"), index_col=0)
        test_df = test_df.loc[:, ~test_df.columns.str.contains('^Unnamed')]
        pred_cols = joblib.load(os.path.join(fp_seed_folder, "pred_cols_"+time_label+".joblib"))
        df_dict[time_label] = {"valid_df": val_df, "test_df": test_df, "pred_cols":pred_cols}
    print("Loaded df_dict!")
    return df_dict

def add_new_ue_to_df_dict(df_dict, seed, model, time_labels=["t+1", "t+2", "t+3"]):
    for time_label in tqdm(time_labels):
        for split in ["valid_df", "test_df"]:
            fp = os.path.join(fp_project_predictions, str(seed), f"{model}_{split[:-3]}_{time_label[-1]}.csv")
            df = pd.read_csv(fp, index_col=0)
            df = df.reset_index(drop=True)
            # find new columns not in current column list
            new_cols = [col for col in df.columns if col not in df_dict[time_label][split].columns]
            if len(new_cols)==0:
                print(f"No new columns in {split}")
                continue
            df_dict[time_label][split] = pd.concat([df_dict[time_label][split], df[new_cols]], axis=1)
    return df_dict

In [ ]:
scaler = load_scaler(fp_downsampled_scaler_file)

# val_df_1 = load_all_predictions(time_label=1, split="valid")
# val_df_2 = load_all_predictions(time_label=2, split="valid")
# val_df_3 = load_all_predictions(time_label=3, split="valid")
    
# test_df_1 = load_all_predictions(time_label=1)
# test_df_2 = load_all_predictions(time_label=2)
# test_df_3 = load_all_predictions(time_label=3)

# df_dict = {
#     "t+1": {"valid_df": val_df_1, "test_df": test_df_1, "pred_cols":pred_cols_1}, 
#     "t+2": {"valid_df": val_df_2, "test_df": test_df_2, "pred_cols":pred_cols_2}, 
#     "t+3": {"valid_df": val_df_3, "test_df": test_df_3, "pred_cols":pred_cols_3}, 
# }

# df_dict = add_new_ue_to_df_dict(df_dict, seed=seed, model="der", time_labels=["t+1", "t+2", "t+3"])

In [ ]:
df_dict = load_df_dict(seed)

#### Calculate KNN Prediction Interval

In [ ]:
def knn_prediction_interval(df_val, df_test, scaler, predictors, pred_cols, pred_label, regressor_label, ue_col, alpha=0.05, seed=seed):
    pi_label = "_knn"
    df_val, df_test = df_val.copy(), df_test.copy()

    unscaled_cols = [col+"_unscaled" for col in pred_cols]
    prediction_cols = [col+pred_label+"_"+regressor_label for col in pred_cols]
    unscaled_pred_cols = [col+"_unscaled" for col in prediction_cols]

    # Unscale the prediction columns
    df_val[unscaled_cols] = scaler.inverse_transform(df_val[pred_cols])
    df_test[unscaled_cols] = scaler.inverse_transform(df_test[pred_cols])
    df_val[unscaled_pred_cols] = scaler.inverse_transform(df_val[prediction_cols])
    df_test[unscaled_pred_cols] = scaler.inverse_transform(df_test[prediction_cols])

    # Set K = sqrt of size of validation set
    k = round(np.sqrt(len(df_val)))

    # Get reconstruction errors
    reconstruction_cols = [col+"_reconstruction_"+regressor_label for col in predictors]
    valid_re = df_val[predictors].values - df_val[reconstruction_cols].values
    test_re = df_test[predictors].values - df_test[reconstruction_cols].values

    timer = Timer(seed=seed)
    timer.start(description="KNN PI")

    # Find neighbours
    n_val = len(valid_re)
    kdtree = KDTree(valid_re)
    dist, ind = kdtree.query(test_re, k=k, workers=-1)

    for col in pred_cols:
        # 1. Val df
        # Get error for each variable
        val_y = df_val[col+"_unscaled"].values.astype('float32')
        val_y_pred = df_val[col+pred_label+"_"+regressor_label+"_unscaled"].values.astype('float32')
        val_pe = np.abs(val_y-val_y_pred)

        # Nearbouring prediction errors
        neighbour_pe = val_pe[ind]

        # PI = 0.95 Percentile of neigbouring errors
        df_test[col+"_"+ue_col+pi_label] = np.quantile(neighbour_pe, np.ceil((k+1)*(1-alpha))/k, method='higher')
        #np.percentile(neighbour_pe, (1-alpha/2)*100, axis=1)

    timer.end()
    
    return df_test


In [ ]:
ue_dict = {
    "RUE": {
        "pred_label": "_direct", "ue_col": "rue", 
    },
}    

# Apply knn_prediction_interval to all RUE
for time_label, time_info in df_dict.items():
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    ue_label = "rue"
    for ue_label, ue_info in ue_dict.items():
        pred_label, ue_col = ue_info["pred_label"], ue_info["ue_col"]
        df_dict[time_label]["test_df"] = knn_prediction_interval(
            df_val=df_dict[time_label]["valid_df"], df_test=df_dict[time_label]["test_df"], predictors=predictors, pred_cols=pred_cols, 
            pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, scaler=scaler)

save_df_dict(df_dict=df_dict, seed=seed)

In [ ]:
[col for col in df_dict["t+1"]["valid_df"].columns if "direct" in col]

#### Calculate Weighted Percentile Prediction Interval

In [ ]:
def weighted_percentile_original(values, weights, percentile): # both are matrices
    n_test = len(values)
    pis = []
    for i in tqdm(range(n_test), total=n_test, leave=False):
        df = pd.DataFrame({'v': values[i], 'w': weights[i]})
        # display(df)
        calc = wc.Calculator('w')  # w designates weight
        pi = calc.quantile(df, 'v', percentile)
        pis.append(pi)
    return pis

def weighted_percentile_matrix(values, weights, percentile):
    ix = np.argsort(values, axis=-1) # sort within each 
    values = np.take_along_axis(values, ix, axis=-1) # sort data
    weights = np.take_along_axis(weights, ix, axis=-1) # sort weights
    cdf = (np.cumsum(weights, axis=-1) - 0.5 * weights) / np.sum(weights, axis=-1, keepdims=True) # 'like' a CDF function
    return [np.interp(percentile, cdf[i], values[i]) for i in range(values.shape[0])]

def weighted_prediction_interval(df_val, df_test, scaler, predictors, pred_cols, pred_label, regressor_label, ue_col, alpha=0.05, seed=seed):
    pi_label = "_weighted"
    df_val, df_test = df_val.copy(), df_test.copy()

    unscaled_cols = [col+"_unscaled" for col in pred_cols]
    prediction_cols = [col+pred_label+"_"+regressor_label for col in pred_cols]
    unscaled_pred_cols = [col+"_unscaled" for col in prediction_cols]

    # Unscale the prediction columns
    df_val[unscaled_cols] = scaler.inverse_transform(df_val[pred_cols])
    df_test[unscaled_cols] = scaler.inverse_transform(df_test[pred_cols])
    df_val[unscaled_pred_cols] = scaler.inverse_transform(df_val[prediction_cols])
    df_test[unscaled_pred_cols] = scaler.inverse_transform(df_test[prediction_cols])

    # Set K = sqrt of size of validation set
    n_val = len(df_val)
    n_test = len(df_test)
    n_feat = len(prediction_cols)
    k = round(np.sqrt(n_val))

    # Get reconstruction errors
    reconstruction_cols = [col+"_reconstruction"+"_"+regressor_label for col in predictors]
    valid_re = df_val[predictors].values - df_val[reconstruction_cols].values
    test_re = df_test[predictors].values - df_test[reconstruction_cols].values

    timer = Timer(seed=seed)
    timer.start(description="Weighted PI")

    # Mahalanobis distance = euclidan distance after pc with whitening
    # - https://stackoverflow.com/questions/69811792/mahalanobis-distance-not-equal-to-euclidean-distance-after-pca
    re_scaler = StandardScaler()
    pca = PCA(whiten=True)
    valid_re = re_scaler.fit_transform(valid_re)
    valid_re = pca.fit_transform(valid_re)
    test_re = re_scaler.transform(test_re)
    test_re = pca.transform(test_re)

    # Find neighbours
    sigma = 10
    kdtree = KDTree(valid_re)
    mahalanobis_dist, ind = kdtree.query(test_re, k=k, workers=-1, p=2)
    dist = mahalanobis_dist/np.sqrt(n_feat) # already sorted by distance (nearest first; ascending order of distance)
    weights = np.exp(-np.square(dist)/(2*sigma**2)) # descending order of weights
    print(np.sum(np.sum(weights, axis=1)==0))
    print(np.mean(np.var(weights, axis=1)))
    modified_alpha = np.ceil((k+1)*(1-alpha))/k

    for col in tqdm(pred_cols):
        # 1. Val df
        # Get error for each variable
        val_y = df_val[col+"_unscaled"].values.astype('float32')
        val_y_pred = df_val[col+pred_label+"_"+regressor_label+"_unscaled"].values.astype('float32')
        val_pe = np.abs(val_y-val_y_pred)

        # Nearbouring prediction errors
        neighbour_pe = val_pe[ind]

        # # PI = 0.95 Weighted Percentile of neigbouring errors
        # pis = []
        # for i in tqdm(range(n_test), total=n_test, leave=False):
        #     df = pd.DataFrame({'v': neighbour_pe[i], 'w': weights[i]})
        #     # display(df)
        #     calc = wc.Calculator('w')  # w designates weight
        #     pi = calc.quantile(df, 'v', modified_alpha)
        #     pis.append(pi)
        pis = weighted_percentile_matrix(values=neighbour_pe, weights=weights, percentile=modified_alpha)
        df_test[col+"_"+ue_col+pi_label] = pis

    timer.end()
    
    return df_test

In [ ]:
ue_dict = {
    "RUE": {
        "pred_label": "_direct", "ue_col": "rue", 
    },
}    

# Apply weighted_prediction_interval to all RUE
for time_label, time_info in df_dict.items():
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    ue_label = "rue"
    for ue_label, ue_info in ue_dict.items():
        pred_label, ue_col = ue_info["pred_label"], ue_info["ue_col"]
        df_dict[time_label]["test_df"] = weighted_prediction_interval(
            df_val=df_dict[time_label]["valid_df"], df_test=df_dict[time_label]["test_df"], predictors=predictors, pred_cols=pred_cols, 
            pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, scaler=scaler)

save_df_dict(df_dict=df_dict, seed=seed)

#### Calculate Conditional Gaussian Prediction Interval

In [ ]:
class ConditionalGaussianDistribution:
    def __init__(self, Y, X):
        self.Y = Y # Assume shape = (num_samples, num_output)
        self.X = X # Assume shape = (num_samples, num_inputs)
        self.YX = np.concatenate((Y, X), axis=-1) # Assume shape = (num_samples, num_output+num_inputs)
        self.num_Y = self.Y.shape[1]
        self.num_X = self.X.shape[1]

        # Fitting
        self.esti_mean_YX = np.mean(self.YX, axis = 0) # (num_output+num_inputs,)
        self.esti_covariance_matrix_YX = np.cov(self.YX.T, ddof=1) # (num_output+num_inputs, num_output+num_inputs)
        
        # Useful stats
        self.mu_Y = self.esti_mean_YX[:self.num_Y]
        self.mu_X = self.esti_mean_YX[self.num_Y:]
        self.cov_XX = self.esti_covariance_matrix_YX[self.num_Y:, self.num_Y:]
        self.cov_YY = self.esti_covariance_matrix_YX[:self.num_Y, :self.num_Y]
        self.cov_YX = self.esti_covariance_matrix_YX[:self.num_Y, self.num_Y:]
        self.cov_XY = self.esti_covariance_matrix_YX[self.num_Y:, :self.num_Y]
        
    def get_conditional_mean(self, alpha):
        # alpha = (num_samples, self.num_X)
        return (
            self.mu_Y + 
            np.linalg.multi_dot((
                self.cov_YX,
                np.linalg.inv(self.cov_XX),
                (alpha - self.mu_X).T))
        ).flatten()
        
    def get_conditional_cov(self):
        return (
            self.cov_YY - 
            np.linalg.multi_dot((
                self.cov_YX,
                np.linalg.inv(self.cov_XX), 
                self.cov_XY))
        ).flatten()[0]

def cond_gauss_prediction_interval(df_val, df_test, scaler, predictors, pred_cols, pred_label, regressor_label, ue_col, alpha=0.05, seed=seed):
    pi_label = "_cond_gauss"
    df_val, df_test = df_val.copy(), df_test.copy()

    unscaled_cols = [col+"_unscaled" for col in pred_cols]
    prediction_cols = [col+pred_label+"_"+regressor_label for col in pred_cols]
    unscaled_pred_cols = [col+"_unscaled" for col in prediction_cols]

    # Unscale the prediction columns
    df_val[unscaled_cols] = scaler.inverse_transform(df_val[pred_cols])
    df_test[unscaled_cols] = scaler.inverse_transform(df_test[pred_cols])
    df_val[unscaled_pred_cols] = scaler.inverse_transform(df_val[prediction_cols])
    df_test[unscaled_pred_cols] = scaler.inverse_transform(df_test[prediction_cols])

    # Get reconstruction errors
    reconstruction_cols = [col+"_reconstruction_"+regressor_label for col in predictors]
    valid_re = df_val[predictors].values - df_val[reconstruction_cols].values
    test_re = df_test[predictors].values - df_test[reconstruction_cols].values

    n_val = len(df_val)

    timer = Timer(seed=seed)
    timer.start(description="Conditional PI")

    for col in tqdm(pred_cols):
        # 1. Val df
        # Get error for each variable
        val_y = df_val[col+"_unscaled"].values.astype('float32')
        val_y_pred = df_val[col+pred_label+"_"+regressor_label+"_unscaled"].values.astype('float32')
        val_pe = val_y-val_y_pred

        # Stack both pe and reconstruction error to fit the cond gaussian model
        val_data = np.hstack((np.expand_dims(val_pe, axis=1), valid_re))
        
        # Parameter Estimation on Validation Set
        uncertainty_distribution = ConditionalGaussianDistribution(
            Y=np.expand_dims(val_pe, axis=1), 
            X= valid_re
        )
        esti_conditional_mean_Y = uncertainty_distribution.get_conditional_mean(test_re)
        esti_conditional_std_Y = np.sqrt(uncertainty_distribution.get_conditional_cov())
        
        # 1-alpha/2 -> Because Gaussian is symmetrical
        df_test[col+"_"+ue_col+pi_label] = norm.ppf(1-alpha/2, loc=esti_conditional_mean_Y, scale=esti_conditional_std_Y).flatten()

    timer.end()
    
    return df_test


In [ ]:
ue_dict = {
    "RUE": {
        "pred_label": "_direct", "ue_col": "rue", 
    },
}    

# Apply cond_gauss_prediction_interval to all RUE
for time_label, time_info in df_dict.items():
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    ue_label = "rue"
    for ue_label, ue_info in ue_dict.items():
        pred_label, ue_col = ue_info["pred_label"], ue_info["ue_col"]
        df_dict[time_label]["test_df"] = cond_gauss_prediction_interval(
            df_val=df_dict[time_label]["valid_df"], df_test=df_dict[time_label]["test_df"], predictors=predictors, pred_cols=pred_cols, 
            pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, scaler=scaler)

save_df_dict(df_dict=df_dict, seed=seed)

#### Calculate Gaussian Corpula Prediction Interval

In [ ]:
EPSILON = np.finfo(np.float32).eps

class ConditionalGaussianDistribution:
    def __init__(self, Y, X):
        self.Y = Y # Assume shape = (num_samples, num_output)
        self.X = X # Assume shape = (num_samples, num_inputs)
        self.YX = np.concatenate((Y, X), axis=-1) # Assume shape = (num_samples, num_output+num_inputs)
        self.num_Y = self.Y.shape[1]
        self.num_X = self.X.shape[1]

        # Fitting
        self.esti_mean_YX = np.mean(self.YX, axis = 0) # (num_output+num_inputs,)
        self.esti_covariance_matrix_YX = np.cov(self.YX.T, ddof=1) # (num_output+num_inputs, num_output+num_inputs)
        
        # Useful stats
        self.mu_Y = self.esti_mean_YX[:self.num_Y]
        self.mu_X = self.esti_mean_YX[self.num_Y:]
        self.cov_XX = self.esti_covariance_matrix_YX[self.num_Y:, self.num_Y:]
        self.cov_YY = self.esti_covariance_matrix_YX[:self.num_Y, :self.num_Y]
        self.cov_YX = self.esti_covariance_matrix_YX[:self.num_Y, self.num_Y:]
        self.cov_XY = self.esti_covariance_matrix_YX[self.num_Y:, :self.num_Y]
        
    def get_conditional_mean(self, alpha):
        # alpha = (num_samples, self.num_X)
        return (
            self.mu_Y + 
            np.linalg.multi_dot((
                self.cov_YX,
                np.linalg.inv(self.cov_XX),
                (alpha - self.mu_X).T))
        ).flatten()
        
    def get_conditional_cov(self):
        return (
            self.cov_YY - 
            np.linalg.multi_dot((
                self.cov_YX,
                np.linalg.inv(self.cov_XX), 
                self.cov_XY))
        ).flatten()[0]

class GaussianCopula:
    def __init__(self, X):
        self.X = X
        self.n_feat = X.shape[1]

        # Get all CDFs
        self.cdfs = []
        for i_feat in range(self.n_feat):
            self.cdfs.append(stats.ecdf(X[:, i_feat]))

    def add_y(self, y):
        self.y = y
        self.y_cdf = stats.ecdf(self.y)
        
    def X_to_V(self, cur_X):
        X_Vi = np.empty(cur_X.shape)
        for i_feat in range(self.n_feat):
            Fi = self.cdfs[i_feat].cdf.evaluate(cur_X[:, i_feat]).clip(EPSILON, 1 - EPSILON)
            X_Vi[:,i_feat] = norm.ppf(Fi)
        return X_Vi

    def Y_to_V(self, cur_y):
        y_Fi = self.y_cdf.cdf.evaluate(cur_y).clip(EPSILON, 1 - EPSILON)
        y_Vi = norm.ppf(y_Fi)
        return y_Vi

    def V_to_Y(self, cur_y_Vi):
        y_Fi = norm.cdf(cur_y_Vi)
        y = np.percentile(self.y, y_Fi*100, axis=0, method="higher")
        return y

def gauss_copula_prediction_interval(df_val, df_test, scaler, predictors, pred_cols, pred_label, regressor_label, ue_col, alpha=0.05, seed=seed):
    pi_label = "_gauss_copula"
    df_val, df_test = df_val.copy(), df_test.copy()

    unscaled_cols = [col+"_unscaled" for col in pred_cols]
    prediction_cols = [col+pred_label+"_"+regressor_label for col in pred_cols]
    unscaled_pred_cols = [col+"_unscaled" for col in prediction_cols]

    # Unscale the prediction columns
    df_val[unscaled_cols] = scaler.inverse_transform(df_val[pred_cols])
    df_test[unscaled_cols] = scaler.inverse_transform(df_test[pred_cols])
    df_val[unscaled_pred_cols] = scaler.inverse_transform(df_val[prediction_cols])
    df_test[unscaled_pred_cols] = scaler.inverse_transform(df_test[prediction_cols])

    # Get reconstruction errors
    reconstruction_cols = [col+"_reconstruction"+"_"+regressor_label for col in predictors]
    valid_re = df_val[predictors].values - df_val[reconstruction_cols].values
    test_re = df_test[predictors].values - df_test[reconstruction_cols].values

    timer = Timer(seed=seed)
    timer.start(description="Copula PI")

    # Convert reconstruction and prediction error to V
    gc = GaussianCopula(valid_re)
    valid_re = gc.X_to_V(valid_re)
    test_re = gc.X_to_V(test_re)

    n_val = len(df_val)

    for col in tqdm(pred_cols):
        # 1. Val df
        # Get error for each variable
        val_y = df_val[col+"_unscaled"].values.astype('float32')
        val_y_pred = df_val[col+pred_label+"_"+regressor_label+"_unscaled"].values.astype('float32')
        val_pe = np.abs(val_y-val_y_pred)

        gc.add_y(val_pe)
        val_pe = gc.Y_to_V(val_pe)

        # Parameter Estimation on Validation Set
        uncertainty_distribution = ConditionalGaussianDistribution(
            Y=np.expand_dims(val_pe, axis=1), 
            X= valid_re
        )
        esti_conditional_mean_Y = uncertainty_distribution.get_conditional_mean(test_re)
        esti_conditional_std_Y = np.sqrt(uncertainty_distribution.get_conditional_cov())

        pi_V = norm.ppf(1-alpha, loc=esti_conditional_mean_Y, scale=esti_conditional_std_Y).flatten()
        # print(pi_V)

        # Convert from V space back to Y
        df_test[col+"_"+ue_col+pi_label] = gc.V_to_Y(pi_V)

    timer.end()
    
    return df_test


In [ ]:
ue_dict = {
    "RUE": {
        "pred_label": "_direct", "ue_col": "rue", 
    },
}    

# Apply weighted_prediction_interval to all RUE
for time_label, time_info in df_dict.items():
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    ue_label = "rue"
    for ue_label, ue_info in ue_dict.items():
        pred_label, ue_col = ue_info["pred_label"], ue_info["ue_col"]
        df_dict[time_label]["test_df"] = gauss_copula_prediction_interval(
            df_val=df_dict[time_label]["valid_df"], df_test=df_dict[time_label]["test_df"], predictors=predictors, pred_cols=pred_cols, 
            pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, scaler=scaler)

save_df_dict(df_dict=df_dict, seed=seed)

In [ ]:
df_dict = load_df_dict(seed)

#### Calculate Conformal Prediction Intervals

In [ ]:
def conformal_prediction_interval(df_val, df_test, scaler, predictors, pred_cols, pred_label, regressor_label, ue_col, alpha=0.05, seed=seed):
    pi_label = "_conformal"
    df_val, df_test = df_val.copy(), df_test.copy()
    unscaled_cols = [col+"_unscaled" for col in pred_cols]
    prediction_cols = [col+pred_label+"_"+regressor_label for col in pred_cols]
    unscaled_pred_cols = [col+"_unscaled" for col in prediction_cols]
    df_val[unscaled_cols] = scaler.inverse_transform(df_val[pred_cols])
    df_test[unscaled_cols] = scaler.inverse_transform(df_test[pred_cols])
    df_val[unscaled_pred_cols] = scaler.inverse_transform(df_val[prediction_cols])
    df_test[unscaled_pred_cols] = scaler.inverse_transform(df_test[prediction_cols])

    timer = Timer(seed=seed)
    timer.start(description="Conformal PI")

    for col in pred_cols:
        # 1. Val df
        # Get error for each variable
        val_y = df_val[col+"_unscaled"].values.astype('float32')
        val_y_pred = df_val[col+pred_label+"_"+regressor_label+"_unscaled"].values.astype('float32')
        val_error = np.abs(val_y-val_y_pred)
    
        # Get uncertainty 
        val_ue = df_val[ue_col].astype('float32')

        # Get scores
        val_scores = val_error/val_ue
        # Get the score quantile
        n = len(df_val)
        qhat = np.quantile(val_scores, np.ceil((n+1)*(1-alpha))/n, method='higher')

        # 2. Test df
        # Get uncertainty 
        test_ue = df_test[ue_col].astype('float32')

        # Caclulate PI
        df_test[col+"_"+ue_col+pi_label] = test_ue*qhat
        
    timer.end()
    
    return df_test

In [ ]:
ue_dict = {
    "RUE": {
        "pred_label": "_direct", "ue_col": "rue", 
    },
    "MC Dropout": {
        "pred_label": "_mean", "ue_col": "mcd", 
    },
    "GPR": {
        "pred_label": "_gpr", "ue_col": "gpr_std_mean", 
    },
    "Infer-Noise": {
        "pred_label": "_infernoise", "ue_col": "infernoise_uncertainty", 
    },
    "BNN": {
        "pred_label": "_bnn", "ue_col": "bnn_uncertainty", 
    },
    "DER": {
        "pred_label": "_der", "ue_col": "der_uncertainty", 
    }
}    

# Apply conformal prediction to all UEs
for time_label, time_info in df_dict.items():
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    for ue_label, ue_info in ue_dict.items():
        pred_label, ue_col = ue_info["pred_label"], ue_info["ue_col"]
        df_dict[time_label]["test_df"] = conformal_prediction_interval(
            df_val=df_dict[time_label]["valid_df"], df_test=df_dict[time_label]["test_df"], predictors=predictors, pred_cols=pred_cols, 
            pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, scaler=scaler)

save_df_dict(df_dict=df_dict, seed=seed)

In [ ]:
df_dict["t+1"]["test_df"]

## Evaluation

### Evaluation Functions

In [ ]:
def get_prediction_performance_table(test_df_dict, ue_dict):
    perf_df_dict = []
    for ue_name, ue_info in ue_dict.items():
        ue_row_dict = {"Model": ue_name}
        pred_label = ue_info["pred_label"]
        for regressor_label, test_df_info in test_df_dict.items():
            test_df = test_df_info["test_df"]
            pred_cols = test_df_info["pred_cols"]
            y_pred_cols = [col+pred_label+"_"+regressor_label for col in pred_cols]
            # ue_cols = [col+ue_label for col in predictors]
            ue_row_dict[regressor_label] = sklearn.metrics.mean_squared_error(test_df[pred_cols], test_df[y_pred_cols])
        perf_df_dict.append(ue_row_dict)
    return pd.DataFrame(perf_df_dict)

def calculate_aurc(ue, loss):
    num_samples = len(ue)
    ue_loss_df = pd.DataFrame({"ue":ue, "loss":loss})
    ue_loss_df = ue_loss_df.sort_values(by="ue", ascending=True)
    ue_loss_df["cumulative_loss"] = ue_loss_df["loss"].expanding().mean()
    ue_loss_df["coverage"] = (np.arange(num_samples)+1)/num_samples
    return auc(ue_loss_df["coverage"], ue_loss_df["cumulative_loss"].values)

def min_max_norm(vec):
    return (vec - vec.min()) / (vec.max() - vec.min())

def remove_outliers(vec):
    # vector within 3 std away from mean
    # data_mean, data_std = np.mean(vec), np.std(vec)
    # num_std = 3
    # return vec[(vec <= data_mean + num_std*data_std) & (vec >= data_mean - num_std*data_std)]

    Q1 = np.percentile(vec, 25, method= 'midpoint') 
    Q3 = np.percentile(vec, 75, method= 'midpoint') 
    IQR = Q3 - Q1 
    # low_lim = Q1 - 1.5 * IQR
    up_lim = Q3 + 1.5 * IQR
    return vec[(vec <= up_lim )]
    
def min_max_norm_wo_outliers(vec):
    vec_wo_outliers = remove_outliers(vec)
    vec_min, vec_max = min(vec_wo_outliers), max(vec_wo_outliers)
    return (vec - vec_min) / (vec_max - vec_min)

def calculate_sigma_risk(ue, loss, thres):
    ue = min_max_norm_wo_outliers(ue)
    return loss[ue<=thres].mean()

def get_ue_performance_table(test_df_dict, ue_dict):
    from scipy.stats import pearsonr
    perf_df_dict = []
    for ue_name, ue_info in ue_dict.items():
        ue_row_dict = {"Model": ue_name}
        pred_label = ue_info["pred_label"]
        ue_col = ue_info["ue_col"]
        for regressor_label, test_df_info in test_df_dict.items():
            test_df = test_df_info["test_df"]
            pred_cols = test_df_info["pred_cols"]
            y_pred_cols = [col+pred_label+"_"+regressor_label for col in pred_cols]
            
            y_true = test_df[pred_cols].values
            y_pred = test_df[y_pred_cols].values
            ues = test_df[ue_col].values
            
            mean_abs_errors = np.mean(np.abs(y_true-y_pred), axis=1)
            
            corr, p_value = pearsonr(ues, mean_abs_errors)
            aurc = calculate_aurc(ues, mean_abs_errors)

            ue_row_dict[regressor_label+" Corr"] = corr
            ue_row_dict[regressor_label+" Pval"] = p_value
            ue_row_dict[regressor_label+" AURC"] = aurc

            for thres in [0.1, 0.2, 0.3, 0.4]:
                ue_row_dict[regressor_label+f" Sigma={thres}"] = calculate_sigma_risk(ues, mean_abs_errors, thres)
            
        perf_df_dict.append(ue_row_dict)
    return pd.DataFrame(perf_df_dict)

def calculate_picp_for_a_feature(actual, lb, ub):
    # PICP (Prediction interval coverage probability)
    # - The percentage of points within the prediction interval; The higher the better. 
    points_within_pi = ((actual >= lb) & (actual <= ub))
    num_points_within_pi = points_within_pi.sum()
    total_num_points = len(actual)
    picp = num_points_within_pi/total_num_points
    return picp

def calculate_pinaw_for_a_feature(actual, lb, ub):
    # Prediction interval normalized average width (PINAW)
    # - The average size of the prediction interval. 
    range_of_underlying_target = actual.max() - actual.min()
    total_num_points = len(actual)
    width_of_pi = ub-lb
    sum_of_widths_of_pi = width_of_pi.sum()
    pinaw = sum_of_widths_of_pi/(total_num_points*range_of_underlying_target)
    return pinaw

def calculate_pinafd_for_a_feature(actual, lb, ub):
    # Prediction interval normalized average failure distance (PINAFD)
    # - The average distance of points outside of the prediction interval to the bounds of the prediction interval. 
    range_of_underlying_target = actual.max() - actual.min()
    points_outside_pi = ((actual < lb) | (actual > ub))
    num_points_outside_pi = points_outside_pi.sum()
    if num_points_outside_pi == 0:
        return 0
    else:
        dist_lb = (lb-actual).abs()
        dist_ub = (ub-actual).abs()
        shortest_dist_to_any_b = np.minimum(dist_lb, dist_ub)
        failure_dist = shortest_dist_to_any_b[points_outside_pi]
        pinafd = failure_dist.sum()/(num_points_outside_pi*range_of_underlying_target)
        return pinafd
    
def calculate_cp_for_a_feature(actual, lb, ub):
    picp = calculate_picp_for_a_feature(actual, lb, ub)
    alpha = 0.05
    delta = alpha/50
    cp = (1-alpha+delta-picp)**2
    return cp
    
    
def calculate_cwfdc_for_a_feature(actual, lb, ub):
    pinaw = calculate_pinaw_for_a_feature(actual, lb, ub)
    pinafd = calculate_pinafd_for_a_feature(actual, lb, ub)
    cp = calculate_cp_for_a_feature(actual, lb, ub)
    rho = 1
    beta = 1000
    cwfdc = pinaw + rho*pinafd + beta * cp
    return cwfdc

def aggregate_metrics_across_feats(df_info, pi_info, time_label, metric_func):
    total_metric = 0
    pred_cols, df_test = df_info["pred_cols"], df_info["test_df"]
    pred_label, ue_col,  pi_label = pi_info["pred_label"], pi_info["ue_col"], pi_info["pi_label"]
    num_feat = len(pred_cols)
    for pred_col in pred_cols:
        pi_col = f"{pred_col}_{ue_col}{pi_label}"
        predicted_col = f"{pred_col}{pred_label}_{time_label}_unscaled"
        pi, pred = df_test[pi_col], df_test[predicted_col]
        total_metric += metric_func(
            actual=df_test[pred_col+"_unscaled"], 
            lb=pred-pi, 
            ub=pred+pi
        )
    av_metric = total_metric/num_feat
    return av_metric

def calculate_metrics(df_dict, pi_dict):
    output_df_list = []
    feature_list = ['SpO2 (%)', 'RESP (bpm)', 'ABPmean (mmHg)', 'HR (bpm)', 'ABPsys (mmHg)', 'ABPdias (mmHg)']
    for time_label, df_info in df_dict.items():
        for pi_label, pi_info in pi_dict.items():
            picp = aggregate_metrics_across_feats(df_info, pi_info, time_label, metric_func=calculate_picp_for_a_feature)
            pinaw = aggregate_metrics_across_feats(df_info, pi_info, time_label, metric_func=calculate_pinaw_for_a_feature)
            pinafd = aggregate_metrics_across_feats(df_info, pi_info, time_label, metric_func=calculate_pinafd_for_a_feature)
            cp = aggregate_metrics_across_feats(df_info, pi_info, time_label, metric_func=calculate_cp_for_a_feature)
            cwfdc = aggregate_metrics_across_feats(df_info, pi_info, time_label, metric_func=calculate_cwfdc_for_a_feature)
            output_df_list.append({
                "Time Horizon":time_label,
                "Method":pi_label,
                "PICP":picp,
                "PINAW":pinaw,
                "PINAFD":pinafd,
                "CP":cp,
                "CWFDC":cwfdc
            })
            
    output_df = pd.DataFrame(output_df_list)
    return output_df
    

### Load Predictions

In [ ]:
def load_scaler(fp_downsampled_scaler_file):
    import pickle 
    with open(fp_downsampled_scaler_file, 'rb') as handle:
        scaler = pickle.load(handle)
    return scaler

# Load all predictions
def load_df_dict(seed, time_labels=["t+1", "t+2", "t+3"]):
    df_dict = {}
    for time_label in tqdm(time_labels):
        fp_seed_folder = os.path.join(fp_project_pi_predictions, str(seed))
        val_df = pd.read_csv(os.path.join(fp_seed_folder, "val_"+time_label+".csv"))
        test_df = pd.read_csv(os.path.join(fp_seed_folder, "test_"+time_label+".csv"))
        pred_cols = joblib.load(os.path.join(fp_seed_folder, "pred_cols_"+time_label+".joblib"))
        df_dict[time_label] = {"valid_df": val_df, "test_df": test_df, "pred_cols":pred_cols}
    print("Loaded df_dict!")
    return df_dict

In [ ]:
scaler = load_scaler(fp_downsampled_scaler_file)

df_dict = load_df_dict(seed)

### Prediction Performance Table

In [ ]:
ue_dict = {
    "RUE": {
        "pred_label": "_direct", "ue_col": "rue", 
    },
    "Infer-Noise": {
        "pred_label": "_infernoise", "ue_col": "infernoise_uncertainty", 
    },
    "MC Dropout": {
        "pred_label": "_mean", "ue_col": "mcd", 
    },
    "GPR": {
        "pred_label": "_gpr", "ue_col": "gpr_std_mean", 
    },
    "BNN": {
        "pred_label": "_bnn", "ue_col": "bnn_uncertainty", 
    },
    "DER": {
        "pred_label": "_der", "ue_col": "der_uncertainty", 
    },
}    
pred_perf_df = get_prediction_performance_table(df_dict, ue_dict)
display(pred_perf_df)
pred_perf_df.to_csv(os.path.join(fp_project_model_evaluations, str(seed), "pred_perf.csv"))

### Compare UE Performance

In [ ]:
ue_perf_df = get_ue_performance_table(df_dict, ue_dict)
display(ue_perf_df)
ue_perf_df.to_csv(os.path.join(fp_project_model_evaluations, str(seed), "ue_perf.csv"))

In [ ]:
def display_ue_perf(ue_perf_df):
    for i in range(3):
        column_indices = [0] + list(range(1+i*7, 1+(i+1)*7))
        display(ue_perf_df.iloc[:,column_indices].set_index("Model"))
display_ue_perf(ue_perf_df)

### Compare PI Performance

In [ ]:
pi_dict = {
    "RUE Gaussian Copula": {
        "pred_label": "_direct", "ue_col": "rue", "pi_label": "_gauss_copula"
    },
    "RUE Conditional Gaussian": {
        "pred_label": "_direct", "ue_col": "rue", "pi_label": "_cond_gauss"
    },
    "RUE Weighted": {
        "pred_label": "_direct", "ue_col": "rue", "pi_label": "_weighted"
    },
    "RUE KNN": {
        "pred_label": "_direct", "ue_col": "rue", "pi_label": "_knn"
    },
    "RUE Conformal": {
        "pred_label": "_direct", "ue_col": "rue", "pi_label": "_conformal"
    },
    "Infer-Noise Conformal": {
        "pred_label": "_infernoise", "ue_col": "infernoise_uncertainty", "pi_label": "_conformal"
    },
    "MC Dropout Conformal": {
        "pred_label": "_mean", "ue_col": "mcd", "pi_label": "_conformal"
    },
    "GPR Conformal": {
        "pred_label": "_gpr", "ue_col": "gpr_std_mean", "pi_label": "_conformal"
    },
    "BNN Conformal": {
        "pred_label": "_bnn", "ue_col": "bnn_uncertainty", "pi_label": "_conformal"
    },
    "DER Conformal": {
        "pred_label": "_der", "ue_col": "der_uncertainty", "pi_label": "_conformal"
    },
}   

pi_stats = calculate_metrics(df_dict, pi_dict)
# pi_stats.to_csv(os.path.join(fp_project_model_evaluations, str(seed), "pi_perf.csv"))
pi_stats

### Paper Graphs

#### Line Graph

In [ ]:
def plot_predictions_with_pi(df_dict, pi_dict, scaler, seed, record="055n", dpi=300, save_figs=False):
    fp_fig_folder = os.path.join(fp_project_model_evaluations, str(seed), "pi_line_graphs")
    create_folder(fp_fig_folder)

    features = scaler.feature_names_in_

    num_rows = len(df_dict)
    num_cols = len(df_dict["t+1"]["pred_cols"])
    
    # Plots in multiples of three
    for pi_name, pi_info in pi_dict.items():
        print(pi_name, ":")
        pred_label, ue_col, pi_label = pi_info["pred_label"], pi_info["ue_col"], pi_info["pi_label"]

        # Plot for all three time intervals
        fig, axes = plt.subplots(num_rows, num_cols, dpi=dpi, figsize=(2.5*num_cols, 2*num_rows))
        
        for i, (regressor_label, test_df_info) in enumerate(df_dict.items()):
            test_df = test_df_info["test_df"]
            test_record_df = test_df.loc[test_df["record"]==record]
            pred_cols = test_df_info["pred_cols"]

            #     # Sort columns for display
            zipped_cols = list(zip(features, pred_cols))
            zipped_cols = sorted(zipped_cols, key = lambda x: x[0])
    
            for j, (feature, pred_col) in enumerate(zipped_cols):
                y_pred_col = pred_col+pred_label+"_"+regressor_label

                index = test_record_df["target_index"]
                y_true = test_record_df[pred_col+"_unscaled"].values
                y_pred = test_record_df[y_pred_col+"_unscaled"].values
                pi = test_record_df[pred_col+"_"+ue_col+pi_label].values
                
                # Plot predictions and their CI
                axes[i,j].plot(index, y_true, color="red")
                axes[i,j].plot(index, y_pred, color="green")
                axes[i,j].fill_between(
                    index, (y_pred-pi), (y_pred+pi), 
                    color='green', alpha=0.3
                )  
                if j!=0:
                    axes[i,j].set_ylabel(feature)
                else:
                    axes[i,j].set_ylabel(regressor_label+"\n"+feature)
        fig.suptitle(pi_name)
        plt.tight_layout()
        if save_figs:
            plt.savefig(os.path.join(fp_fig_folder, ue_col+pi_label+".jpg"), bbox_inches="tight")
        plt.show()

In [ ]:
# function to calculate pis
plot_predictions_with_pi(df_dict, pi_dict, scaler, seed=seed, dpi=128, save_figs=False)

#### Parallel Coordinate Plot

In [ ]:
def plot_pi_metrics_parallel_coordinate(pi_stats, seed, categories=["CP", "PINAW", "PINAFD"], save_fig=False):
    from matplotlib import cm
    from matplotlib.colors import rgb2hex
    import seaborn as sns
    
    pi_stats = pi_stats.copy().sort_values(by=["Time Horizon", "Method"])
    num_cat = len(categories)
    
    rue_colors = sns.color_palette('spring', n_colors=4)
    other_colors = sns.color_palette('winter', n_colors=3)
    
    # color_list =  other_colors + rue_colors
    color_list = sns.color_palette('rainbow', n_colors=pi_stats["Method"].nunique())
    print(pi_stats["Method"].unique())
    # color_list = [
    #     '#1f77b4',  # muted blue
    #     '#ff7f0e',  # safety orange
    #     '#2ca02c',  # cooked asparagus green
    #     '#d62728',  # brick red
    #     '#9467bd',  # muted purple
    #     '#8c564b',  # chestnut brown
    #     '#e377c2',  # raspberry yogurt pink
    # #     '#7f7f7f',  # middle gray
    # #     '#bcbd22',  # curry yellow-green
    # #     '#17becf'   # blue-teal
    # ]
    
    fig, axes = plt.subplots(1, num_cat, sharey=True, dpi=300, figsize=(num_cat*3, 3))
    xticks = range(num_cat)
    
    for i, (time_horizon_label, time_df) in enumerate(pi_stats.groupby("Time Horizon")):
        time_df[categories] = (time_df[categories]-time_df[categories].min())/(time_df[categories].max()-time_df[categories].min())
        for j, (method_label, method_df) in enumerate(time_df.groupby("Method")):
            # print(method_df[categories].values.flatten())
            axes[i].plot(xticks, method_df[categories].values.flatten(), label=method_label if i==0 else None, linestyle="--" if "RUE" not in method_label else "-")
            axes[i].set_xticks(xticks, categories)
        axes[i].set_title(time_horizon_label)

    reorder=lambda hl,nc:(sum((lis[i::nc]for i in range(nc)),[])for lis in hl)
    h_l = axes[0].get_legend_handles_labels()
    fig.legend(*reorder(h_l, 5), loc='upper center', bbox_to_anchor=(0.5, 0), ncol=5, fontsize=8)
    plt.tight_layout()
    if save_fig:
        fp_fig_folder = os.path.join(fp_project_model_evaluations, str(seed))
        plt.savefig(os.path.join(fp_fig_folder, "parallel_coord.jpg"), bbox_inches="tight")
    

In [ ]:
plot_pi_metrics_parallel_coordinate(pi_stats, seed=seed, categories=["CP", "PINAW", "PINAFD"], save_fig=True)

#### UE-Error Scatter Plot

In [ ]:
def create_subtitle(fig: plt.Figure, grid: SubplotSpec, title: str):
    "Sign sets of subplots with title"
    row = fig.add_subplot(grid)
    # the '\n' is important
    row.set_title(f'{title}\n', fontweight='semibold', pad=0, y=0.975)
    # hide subplot
    row.set_frame_on(False)
    row.axis('off')

# Scatter plot of rows of t+1, t+2... t+3
# - In each row, we have the scatterplots of each UE
def get_ue_loss_scatterplot(test_df_dict, ue_dict, seed, nbins=10, dpi=300, save_fig=False):
    eqn_label_fs = 7
    line_col = 'black'
    point_color = "#0090C1"
    point_size = 75
    point_alpha = 0.5
    marker = "."
    formatting_dict = dict(color=point_color, s=point_size, alpha=point_alpha, marker=marker, edgecolors='none')
    
    bin_width = int(100/nbins)
    num_cols = len(ue_dict)
    num_rows = len(test_df_dict) # num intervals
    #, constrained_layout=True
    # fig, axes = plt.subplots(num_rows*2, num_cols, dpi=dpi, figsize=(num_cols*4, 1.5*2*num_rows)) # , sharey="row", sharex="col"
    fig = plt.figure(dpi=dpi, figsize=(num_cols*3, 2*2*num_rows))
    grid = fig.add_gridspec(num_rows, 1, height_ratios=[1 for i in range(num_rows)], hspace=0.3)
    
    # For all time intervals 
    for time_i ,(regressor_label, test_df_info) in enumerate(test_df_dict.items()):
        cur_gs = grid[time_i].subgridspec(2, num_cols, wspace=0, hspace=0.1)
        axes = cur_gs.subplots(sharey='row', sharex='col')
        test_df = test_df_info["test_df"]
        pred_cols = test_df_info["pred_cols"]
        y_true = test_df[pred_cols].values

        create_subtitle(fig, grid[time_i, ::], regressor_label)
        # For each ue 
        for ue_i, (ue_type, ue_info) in enumerate(ue_dict.items()):
            
            ue_df = test_df.copy()
            
            pred_label = ue_info["pred_label"]
            ue_col = ue_info["ue_col"]

            # Get error and UE cols
            y_pred_cols = [col+pred_label+"_"+regressor_label for col in pred_cols]
            y_pred = test_df[y_pred_cols].values
            ue = test_df[ue_col].values
            loss = np.mean(np.abs(y_true-y_pred), axis=1)
            ue_df["loss"] = loss

            # Normalise UE
            ue = min_max_norm_wo_outliers(ue)
            ue_df[ue_col] = ue
            max_range = round(max(ue), 1)
            bin_edges = np.array([0]+list(np.linspace(bin_width, int(max_range*100), num=int(max_range*10))/100))
            
            # Plot scatter
            scatter_ax = axes[0, ue_i]
            scatter_ax.axvspan(0, bin_width/100, color='grey', alpha=0.2, linewidth=0)
            scatter_ax.scatter(ue, loss, **formatting_dict)
            # Plot line
            m, c = np.polyfit(ue, loss, 1) 
            ue = np.sort(ue)
            scatter_ax.plot(ue, m*ue+c, color=line_col, linestyle='-', label=f'y = {m:.3f}x + {c:.3f}', linewidth=1.5)
            scatter_ax.legend(fontsize=eqn_label_fs)
            if ue_i == 0:
                axes[0, ue_i].set_ylabel("MAE")
                
            # Plot Barplot
            ue_df['bin'] = pd.cut(ue_df[ue_col], bins=bin_edges, labels=bin_edges[1:], include_lowest=True, right=True)
            grouped = ue_df.groupby("bin", observed=False)
            grouped_loss = grouped["loss"].mean()
            # Comparison of losses
            bar_ax = axes[1, ue_i]
            bar_ax.bar(
                bin_edges[1:]-bin_width/100/2, grouped_loss, width=bin_width/100*0.9, color="#0090C1")
            ticks = list(axes[1, ue_i].get_yticks(minor=True)) + [grouped_loss[0.1]]
            bar_ax.set_yticks(ticks, minor=True)
            if ue_i == 0:
                bar_ax.set_ylabel("Mean MAE")
            bar_ax.set_xlabel(ue_type)

    if save_fig:
        fp_fig_folder = os.path.join(fp_project_model_evaluations, str(seed))
        plt.savefig(os.path.join(fp_fig_folder, "ue_error_scatter.jpg"), bbox_inches="tight")


In [ ]:
ue_dict = {
    "RUE": {
        "pred_label": "_direct", "ue_col": "rue", 
    },
    "Infer-Noise": {
        "pred_label": "_infernoise", "ue_col": "infernoise_uncertainty", 
    },
    "MC Dropout": {
        "pred_label": "_mean", "ue_col": "mcd", 
    },
    "GPR": {
        "pred_label": "_gpr", "ue_col": "gpr_std_mean", 
    },
    "BNN": {
        "pred_label": "_bnn", "ue_col": "bnn_uncertainty", 
    },
    "DER": {
        "pred_label": "_der", "ue_col": "der_uncertainty", 
    },
}    
get_ue_loss_scatterplot(df_dict, ue_dict, seed=seed, dpi=300, save_fig=True)

#### Starplot

In [ ]:
def plot_pi_metrics_starplot(pi_stats, categories=pi_stats.columns.tolist()[2:-2]):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    from plotly.colors import hex_to_rgb
    fig = make_subplots(
        rows=1, cols=pi_stats["Time Horizon"].nunique(),
        specs=[[{"type": "scatterpolar"}, {"type": "scatterpolar"}, {"type": "scatterpolar"}]], 
        subplot_titles=pi_stats["Time Horizon"].unique()
    )
    pi_stats = pi_stats.copy()
    color_list = [
        '#1f77b4',  # muted blue
        '#ff7f0e',  # safety orange
        '#2ca02c',  # cooked asparagus green
        '#d62728',  # brick red
        '#9467bd',  # muted purple
        '#8c564b',  # chestnut brown
        '#e377c2',  # raspberry yogurt pink
    #     '#7f7f7f',  # middle gray
    #     '#bcbd22',  # curry yellow-green
    #     '#17becf'   # blue-teal
    ]
    
    for i, (time_horizon_label, time_df) in enumerate(pi_stats.groupby("Time Horizon")):
        # time_df[categories] = (time_df[categories]-time_df[categories].min())/(time_df[categories].max()-time_df[categories].min())
        time_df[categories] = time_df[categories]/time_df[categories].max()
        # time_df[categories] = np.log(time_df[categories])
        for j, (method_label, method_df) in enumerate(time_df.groupby("Method")):
            cur_col = list(hex_to_rgb(color_list[j]))
            cur_fill_col = f'rgba({cur_col[0]}, {cur_col[1]}, {cur_col[2]}, 0.3)'
            cur_outline_col = f'rgba({cur_col[0]}, {cur_col[1]}, {cur_col[2]}, 0.8)'
            fig.append_trace(
                go.Scatterpolar(
                      r=method_df[categories].values[0],
                      theta=categories,
                      fill='toself',
                      name=method_label if i==0 else None,
                      fillcolor=cur_fill_col,
                      marker=dict(color=cur_outline_col, line=dict(
                            color=color_list[j],
                            width=1
                        ),
                    ),
                    showlegend=i==0
                ), row=1, col=i+1
            )

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
              visible=False,
            )
        ),
        polar2=dict(
            radialaxis=dict(
              visible=False,
            )
          ),
        polar3=dict(
            radialaxis=dict(
              visible=False,
            )
      ),
      showlegend=True, 
      width=850,  
      height=350,
      title_x=0.5,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            xanchor="center",
            y=-0.5,
            x=0.5,
            entrywidth=180
        )
    )
#     fig.show()
    return fig
        
plot_pi_metrics_starplot(pi_stats, categories=pi_stats.columns.tolist()[3:-1])

## Repeat Experiment

In [ ]:
hp_seed = 2023

def train_rue_n_predictor(seed, data_dict, override_checkpoints, testing):
    # Load Best HP
    all_rue_decoder_best_hp = joblib.load(os.path.join(fp_tuning, str(hp_seed), "all_rue_decoder_best_hp.joblib"))
    with tqdm(data_dict.items(), total=len(data_dict)) as pbar:
        for time_label, time_info_dict in pbar:
            pbar.set_description(time_label)
            fp_model = os.path.join(fp_project_models, str(seed), f"rue_{time_label}")
            # Check if fp_model exists
            if override_checkpoints or not os.path.exists(fp_model):
                # Get best hyperparameter
                best_hp = all_rue_decoder_best_hp[time_label]
                # Train model
                ae_regressor = model_training(
                    best_hp, predictors=predictors, pred_cols=time_info_dict["outputs"], 
                    train_df=time_info_dict["train_df"], valid_df = time_info_dict["valid_df"], seed=seed,
                    batch_size=batch_size, max_epochs=10000 if not testing else 1, verbose=1, patience=20
                ) 
                # Save model
                save_model(
                    model=ae_regressor, name=f"rue_{time_label}", 
                    fp_checkpoints=os.path.join(fp_project_models, str(seed)), override=True)
            else:
                print(f"- Skip {time_label} Training")

def predict_rue_n_predictor(seed, data_dict, override_checkpoints, testing):
    with tqdm(data_dict.items(), total=len(data_dict)) as pbar:
        for time_label, time_info_dict in pbar:
            pbar.set_description(time_label)
            # Load model
            ae_regressor = load_model(name=f"rue_{time_label}", fp_checkpoints=os.path.join(fp_project_models, str(seed)))
            # Check if we have predicted on the validation set
            fp_valid_df = os.path.join(fp_project_predictions, str(seed), f"rue_valid_{time_label[-1]}.csv")
            if override_checkpoints or not os.path.exists(fp_valid_df):
                rue_valid_df = model_test_predictions(
                    ae_regressor, df_train=time_info_dict["train_df"], df_test=time_info_dict["valid_df"], 
                    pred_cols=time_info_dict["outputs"], predictors=predictors, 
                    regressor_label="_"+time_label, pred_min=int(time_label[-1]), T=10 if not testing else 1, seed=seed
                )
                rue_valid_df.to_csv(fp_valid_df)
            else:
                print(f"- Skip {time_label} Validation Prediction")
                
            # Check if we have predicted on the testing set 
            fp_test_df = os.path.join(fp_project_predictions, str(seed), f"rue_test_{time_label[-1]}.csv")
            if override_checkpoints or not os.path.exists(fp_test_df):
                rue_test_df = model_test_predictions(
                    ae_regressor, df_train=time_info_dict["train_df"], df_test=time_info_dict["test_df"], 
                    pred_cols=time_info_dict["outputs"], predictors=predictors, 
                    regressor_label="_"+time_label, pred_min=int(time_label[-1]), T=10 if not testing else 1, seed=seed
                )
                rue_test_df.to_csv(fp_test_df)
            else:
                print(f"- Skip {time_label} Testing Prediction")
            
def train_gpr(seed, data_dict, override_checkpoints, testing):
    with tqdm(data_dict.items(), total=len(data_dict)) as pbar:
        for time_label, time_info_dict in pbar:
            pbar.set_description(time_label)
            fp_model = os.path.join(fp_project_models, str(seed), f"gpr_{time_label[-1]}")
            # Check if fp_model exists
            if override_checkpoints or not os.path.exists(fp_model):
                # Train model
                gpr = model_training_gpr( 
                    predictors=predictors, pred_cols=time_info_dict["outputs"], 
                    train_df=time_info_dict["train_df"], valid_df = time_info_dict["valid_df"], seed=seed
                ) 
                # Save model
                save_model_gpr(model=gpr, name=f"gpr_{time_label[-1]}", fp_checkpoints=os.path.join(fp_project_models, str(seed)))
            else:
                print(f"- Skip {time_label} Training")

def predict_gpr(seed, data_dict, override_checkpoints, testing):
    with tqdm(data_dict.items(), total=len(data_dict)) as pbar:
        for time_label, time_info_dict in pbar:
            pbar.set_description(time_label)
            gpr = load_model_gpr(name=f"gpr_{time_label[-1]}", fp_checkpoints=os.path.join(fp_project_models, str(seed)))
            # Check if we have predicted on the validation set
            fp_valid_df = os.path.join(fp_project_predictions, str(seed), f"gpr_valid_{time_label[-1]}.csv")
            if override_checkpoints or not os.path.exists(fp_valid_df):
                gpr_valid_df = model_test_predictions_gpr(
                    gpr=gpr, df_test=time_info_dict["valid_df"], pred_cols=time_info_dict["outputs"], 
                    predictors=predictors, regressor_label=f"_{time_label}", pred_min=int(time_label[-1]), seed=seed)
                gpr_valid_df.to_csv(fp_valid_df)
            else:
                print(f"- Skip {time_label} Validation Prediction")
                
            # Check if we have predicted on the testing set
            fp_test_df = os.path.join(fp_project_predictions, str(seed), f"gpr_test_{time_label[-1]}.csv")
            if override_checkpoints or not os.path.exists(fp_test_df):
                gpr_test_df = model_test_predictions_gpr(
                    gpr=gpr, df_test=time_info_dict["test_df"], pred_cols=time_info_dict["outputs"], 
                    predictors=predictors, regressor_label=f"_{time_label}", pred_min=int(time_label[-1]), seed=seed)
                gpr_test_df.to_csv(fp_test_df)
            else:
                print(f"- Skip {time_label} Testing Prediction")    
                
def predict_infernoise(seed, data_dict, override_checkpoints, testing):
    # Load best hp for infernoise
    infernoise_hp_dict = joblib.load(os.path.join(fp_tuning, str(hp_seed), "all_infernoise_best_hp.joblib"))
    with tqdm(data_dict.items(), total=len(data_dict)) as pbar:
        for time_label, time_info_dict in pbar:
            pbar.set_description(time_label)
            # Load model
            ae_regressor = load_model(name=f"rue_{time_label}", fp_checkpoints=os.path.join(fp_project_models, str(seed)))
            # Check if we have predicted on the validation set
            fp_valid_df = os.path.join(fp_project_predictions, str(seed), f"infernoise_valid_{time_label[-1]}.csv")
            if override_checkpoints or not os.path.exists(fp_valid_df):
                infernoise_valid_df = infernoise_test_predictions(
                     ae_regressor, test_df=time_info_dict["valid_df"], inputs=predictors, outputs=time_info_dict["outputs"], regressor_label="_"+time_label, 
                     seed=seed, T=10 if not testing else 1, stddev=infernoise_hp_dict[time_label]
                ) 
                infernoise_valid_df.to_csv(fp_valid_df)
            else:
                print(f"- Skip {time_label} Validation Prediction")
                
            # Check if we have predicted on the testing set
            fp_test_df = os.path.join(fp_project_predictions, str(seed), f"infernoise_test_{time_label[-1]}.csv")
            if override_checkpoints or not os.path.exists(fp_test_df):
                infernoise_test_df = infernoise_test_predictions(
                     ae_regressor, test_df=time_info_dict["test_df"], inputs=predictors, outputs=time_info_dict["outputs"], regressor_label="_"+time_label, 
                     seed=seed, T=10 if not testing else 1, stddev=infernoise_hp_dict[time_label]
                ) 
                infernoise_test_df.to_csv(fp_test_df)
            else:
                print(f"- Skip {time_label} Testing Prediction")    

def train_bnn(seed, data_dict, override_checkpoints, testing):
    # Load Best HP
    all_bnn_best_hp = joblib.load(os.path.join(fp_tuning, str(hp_seed), "all_bnn_best_hp.joblib"))
    with tqdm(data_dict.items(), total=len(data_dict)) as pbar:
        for time_label, time_info_dict in pbar:
            pbar.set_description(time_label)
            fp_model = os.path.join(fp_project_models, str(seed), f"bnn_{time_label}.pt")
            # Check if fp_model exists
            if override_checkpoints or not os.path.exists(fp_model):
                # Get best hyperparameter
                bnn_best_hp = all_bnn_best_hp[time_label]
                # Train model
                bnn_model = train_model_w_best_param(
                    best_param=bnn_best_hp, train_df=time_info_dict["train_df"], valid_df=time_info_dict["valid_df"], 
                    feat_cols=predictors, target_cols=time_info_dict["outputs"],
                    epochs=500 if not testing else 1, patience=5, seed=seed, fp_model=fp_model
                )
            else:
                print(f"- Skip {time_label} Training")

def predict_bnn(seed, data_dict, override_checkpoints, testing):
    with tqdm(data_dict.items(), total=len(data_dict)) as pbar:
        for time_label, time_info_dict in pbar:
            pbar.set_description(time_label)
            # Load model
            fp_model = os.path.join(fp_project_models, str(seed), f"bnn_{time_label}.pt")
            bnn_model = torch.load(fp_model)
            # Check if we have predicted on the validation set
            fp_valid_df = os.path.join(fp_project_predictions, str(seed), f"bnn_valid_{time_label[-1]}.csv")
            if override_checkpoints or not os.path.exists(fp_valid_df):
                bnn_valid_df = bnn_model_prediction(
                    bnn_model, time_info_dict["valid_df"], 
                    feat_cols=predictors, target_cols=time_info_dict["outputs"], 
                    T=10 if not testing else 1, seed=seed, regressor_label=time_label)
                bnn_valid_df.to_csv(fp_valid_df)
            else:
                print(f"- Skip {time_label} Validation Prediction")
                
            # Check if we have predicted on the testing set 
            fp_test_df = os.path.join(fp_project_predictions, str(seed), f"bnn_test_{time_label[-1]}.csv")
            if override_checkpoints or not os.path.exists(fp_test_df):
                bnn_test_df = bnn_model_prediction(
                    bnn_model, time_info_dict["test_df"], 
                    feat_cols=predictors, target_cols=time_info_dict["outputs"], 
                    T=10 if not testing else 1, seed=seed, regressor_label=time_label)
                bnn_test_df.to_csv(fp_test_df)
            else:
                print(f"- Skip {time_label} Testing Prediction")

def train_der(seed, data_dict, override_checkpoints, testing):
    # Load Best HP
    all_der_best_hp = joblib.load(os.path.join(fp_tuning, str(hp_seed), "all_der_best_hp.joblib"))
    with tqdm(data_dict.items(), total=len(data_dict)) as pbar:
        for time_label, time_info_dict in pbar:
            pbar.set_description(time_label)
            fp_model = os.path.join(fp_project_models, str(seed), f"der_{time_label}.pt")
            # Check if fp_model exists
            if override_checkpoints or not os.path.exists(fp_model):
                # Get best hyperparameter
                best_hp = all_der_best_hp[time_label]
                # Train model
                der_model, _ = train_der_w_param(
                    **best_hp, train_df=time_info_dict["train_df"], valid_df=time_info_dict["valid_df"], 
                    inputs=predictors, outputs=time_info_dict["outputs"],
                    seed=seed, max_epochs= 500 if not testing else 1, patience=5
                )
                torch.save(der_model, fp_model)
            else:
                print(f"- Skip {time_label} Training")

def predict_der(seed, data_dict, override_checkpoints, testing):
    with tqdm(data_dict.items(), total=len(data_dict)) as pbar:
        for time_label, time_info_dict in pbar:
            pbar.set_description(time_label)
            # Load model
            fp_model = os.path.join(fp_project_models, str(seed), f"der_{time_label}.pt")
            der_model = torch.load(fp_model)
            # Check if we have predicted on the validation set
            fp_valid_df = os.path.join(fp_project_predictions, str(seed), f"der_valid_{time_label[-1]}.csv")
            if override_checkpoints or not os.path.exists(fp_valid_df):
                der_valid_df = der_model_prediction(
                    der_model, test_df=time_info_dict["valid_df"], 
                    feat_cols=predictors, target_cols=time_info_dict["outputs"], 
                    seed=seed, silent=False, regressor_label=time_label)
                der_valid_df.to_csv(fp_valid_df)
            else:
                print(f"- Skip {time_label} Validation Prediction")
                
            # Check if we have predicted on the testing set 
            fp_test_df = os.path.join(fp_project_predictions, str(seed), f"der_test_{time_label[-1]}.csv")
            if override_checkpoints or not os.path.exists(fp_test_df):
                der_test_df = der_model_prediction(
                    der_model, test_df=time_info_dict["test_df"], 
                    feat_cols=predictors, target_cols=time_info_dict["outputs"], 
                    seed=seed, silent=False, regressor_label=time_label)
                der_test_df.to_csv(fp_test_df)
            else:
                print(f"- Skip {time_label} Testing Prediction")
                
def save_df_dict_fast(df_dict, seed):
    fp_df_dict = os.path.join(fp_project_pi_predictions, str(seed), "df_dict.joblib")
    joblib.dump(df_dict, fp_df_dict)

def load_df_dict_fast( seed):
    fp_df_dict = os.path.join(fp_project_pi_predictions, str(seed), "df_dict.joblib")
    return joblib.load(fp_df_dict)

def retrieve_history_file(seed):
    fp_history = os.path.join(fp_project_pi_predictions, str(seed), "df_dict_history.joblib")
    if os.path.exists(fp_history):
        return joblib.load(fp_history)
    else:
        return []

def add_to_pi_history_file(event, seed):
    fp_history = os.path.join(fp_project_pi_predictions, str(seed), "df_dict_history.joblib")
    if not os.path.exists(fp_history):
        history = set(event)
    else:
        history = joblib.load(fp_history)
        history.add(event)
    joblib.dump(history, fp_history)
    
def create_df_dict(seed, data_dict, override_checkpoints, testing):
    event = "creation"
    history = retrieve_history_file(seed)
    if (override_checkpoints) or (event not in history):
        df_dict = {}
        with tqdm(data_dict.items(), total=len(data_dict)) as pbar:
            for time_label, time_info_dict in pbar:
                df_dict[time_label] = {
                    "valid_df": load_all_predictions(time_label=int(time_label[-1]), split="valid", seed=seed),
                    "test_df": load_all_predictions(time_label=int(time_label[-1]), split="test", seed=seed),
                    "pred_cols": time_info_dict["outputs"]
                }
        save_df_dict_fast(df_dict, seed)
        add_to_pi_history_file(event, seed)
    else:
        print(f"- Skip df_dict creation!")

def generate_prediction_interval(seed, data_dict, override_checkpoints, testing, 
                                 event, pi_func, ue_dict):
    history = retrieve_history_file(seed)
    if (override_checkpoints) or (event not in history):
        df_dict = load_df_dict_fast(seed)
        scaler = load_scaler(fp_downsampled_scaler_file)
        for time_label, time_info in df_dict.items():
            val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
            for ue_label, ue_info in ue_dict.items():
                pred_label, ue_col = ue_info["pred_label"], ue_info["ue_col"]
                df_dict[time_label]["test_df"] = pi_func(
                    df_val=df_dict[time_label]["valid_df"], df_test=df_dict[time_label]["test_df"], predictors=predictors, pred_cols=pred_cols, 
                    pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, scaler=scaler, seed=seed
                )
        save_df_dict_fast(df_dict, seed)
        add_to_pi_history_file(event, seed)
    else:
        print(f"- Skip {event} PI!")

def generate_knn_pi(seed, data_dict, override_checkpoints, testing):
    ue_dict = {"RUE": {"pred_label": "_direct", "ue_col": "rue"}} 
    generate_prediction_interval(
        seed, data_dict, override_checkpoints, testing, 
        event="knn", pi_func=knn_prediction_interval, ue_dict=ue_dict)

def generate_weighted_pi(seed, data_dict, override_checkpoints, testing):
    ue_dict = {"RUE": {"pred_label": "_direct", "ue_col": "rue"}} 
    generate_prediction_interval(
        seed, data_dict, override_checkpoints, testing, 
        event="weighted", pi_func=weighted_prediction_interval, ue_dict=ue_dict)

def generate_conditional_pi(seed, data_dict, override_checkpoints, testing):
    ue_dict = {"RUE": {"pred_label": "_direct", "ue_col": "rue"}} 
    generate_prediction_interval(
        seed, data_dict, override_checkpoints, testing, 
        event="conditional", pi_func=cond_gauss_prediction_interval, ue_dict=ue_dict)

def generate_copula_pi(seed, data_dict, override_checkpoints, testing):
    ue_dict = {"RUE": {"pred_label": "_direct", "ue_col": "rue"}} 
    generate_prediction_interval(
        seed, data_dict, override_checkpoints, testing, 
        event="copula", pi_func=gauss_copula_prediction_interval, ue_dict=ue_dict)

def generate_conformal_pi(seed, data_dict, override_checkpoints, testing):
    ue_dict = {
        "RUE": {"pred_label": "_direct", "ue_col": "rue"},
        "MC Dropout": {"pred_label": "_mean", "ue_col": "mcd", },
        "GPR": {"pred_label": "_gpr", "ue_col": "gpr_std_mean", },
        "Infer-Noise": {"pred_label": "_infernoise", "ue_col": "infernoise_uncertainty", },
        "BNN": {"pred_label": "_bnn", "ue_col": "bnn_uncertainty", },
        "DER": {"pred_label": "_der", "ue_col": "der_uncertainty", }
    }   
    generate_prediction_interval(
        seed, data_dict, override_checkpoints, testing, 
        event="conformal", pi_func=conformal_prediction_interval, ue_dict=ue_dict)

def evaluate_model_perf(seed, data_dict, override_checkpoints, testing):
    df_dict = load_df_dict_fast(seed)
    ue_dict = {
        "RUE": {"pred_label": "_direct", "ue_col": "rue"},
        "MC Dropout": {"pred_label": "_mean", "ue_col": "mcd", },
        "GPR": {"pred_label": "_gpr", "ue_col": "gpr_std_mean", },
        "Infer-Noise": {"pred_label": "_infernoise", "ue_col": "infernoise_uncertainty", },
        "BNN": {"pred_label": "_bnn", "ue_col": "bnn_uncertainty", },
        "DER": {"pred_label": "_der", "ue_col": "der_uncertainty", }
    }   
    fp_pred_perf = os.path.join(fp_project_model_evaluations, str(seed), "pred_perf.csv")
    if override_checkpoints or not os.path.exists(fp_pred_perf):
        pred_perf_df = get_prediction_performance_table(df_dict, ue_dict)
        display(pred_perf_df)
        pred_perf_df.to_csv(fp_pred_perf)
    else:
        print(f"- Skip prediction performance evaluation!")

def evaluate_ue_perf(seed, data_dict, override_checkpoints, testing):
    df_dict = load_df_dict_fast(seed)
    ue_dict = {
        "RUE": {"pred_label": "_direct", "ue_col": "rue"},
        "MC Dropout": {"pred_label": "_mean", "ue_col": "mcd", },
        "GPR": {"pred_label": "_gpr", "ue_col": "gpr_std_mean", },
        "Infer-Noise": {"pred_label": "_infernoise", "ue_col": "infernoise_uncertainty", },
        "BNN": {"pred_label": "_bnn", "ue_col": "bnn_uncertainty", },
        "DER": {"pred_label": "_der", "ue_col": "der_uncertainty", }
    }   
    fp_ue_perf = os.path.join(fp_project_model_evaluations, str(seed), "ue_perf.csv")
    if override_checkpoints or not os.path.exists(fp_ue_perf):
        ue_perf_df = get_ue_performance_table(df_dict, ue_dict)
        display(ue_perf_df)
        ue_perf_df.to_csv(fp_ue_perf)
    else:
        print(f"- Skip UE performance evaluation!")

    fp_ue_scatter = os.path.join(fp_project_model_evaluations, str(seed), "ue_error_scatter.jpg")
    if override_checkpoints or not os.path.exists(fp_ue_scatter):
        get_ue_loss_scatterplot(df_dict, ue_dict, seed=seed, dpi=300, save_fig=True)
    else:
        print(f"- Skip UE-Error Scatter Plot!")

def evaluate_pi_perf(seed, data_dict, override_checkpoints, testing):
    override_checkpoints = True
    df_dict = load_df_dict_fast(seed)
    scaler = load_scaler(fp_downsampled_scaler_file)
    pi_dict = {
        "RUE Gaussian Copula": {"pred_label": "_direct", "ue_col": "rue", "pi_label": "_gauss_copula"},
        "RUE Conditional Gaussian": {"pred_label": "_direct", "ue_col": "rue", "pi_label": "_cond_gauss"},
        "RUE Weighted": {"pred_label": "_direct", "ue_col": "rue", "pi_label": "_weighted"},
        "RUE KNN": {"pred_label": "_direct", "ue_col": "rue", "pi_label": "_knn"},
        "RUE Conformal": {"pred_label": "_direct", "ue_col": "rue", "pi_label": "_conformal"},
        "Infer-Noise Conformal": {"pred_label": "_infernoise", "ue_col": "infernoise_uncertainty", "pi_label": "_conformal"},
        "MC Dropout Conformal": {"pred_label": "_mean", "ue_col": "mcd", "pi_label": "_conformal"},
        "GPR Conformal": {"pred_label": "_gpr", "ue_col": "gpr_std_mean", "pi_label": "_conformal"},
        "BNN Conformal": {"pred_label": "_bnn", "ue_col": "bnn_uncertainty", "pi_label": "_conformal"},
        "DER Conformal": {"pred_label": "_der", "ue_col": "der_uncertainty", "pi_label": "_conformal"},
    }   
    fp_pi_perf = os.path.join(fp_project_model_evaluations, str(seed), "pi_perf.csv")
    if override_checkpoints or not os.path.exists(fp_pi_perf):
        pi_perf_df = calculate_metrics(df_dict, pi_dict)
        display(pi_perf_df)
        pi_perf_df.to_csv(fp_pi_perf)
    else:
        pi_perf_df = pd.read_csv(fp_pi_perf, index_col=0)
        print(f"- Skip PI performance evaluation!")

    fp_pi_folder = os.path.join(fp_project_model_evaluations, str(seed), "pi_line_graphs")
    if override_checkpoints or not os.path.exists(fp_pi_folder):
        plot_predictions_with_pi(df_dict, pi_dict, scaler, seed=seed, dpi=300, save_figs=True)
    else:
        print(f"- Skip PI Line Plots!")

    fp_pi_parallel_coord = os.path.join(fp_project_model_evaluations, str(seed), "parallel_coord.jpg")
    if override_checkpoints or not os.path.exists(fp_pi_parallel_coord):
        plot_pi_metrics_parallel_coordinate(pi_perf_df, seed=seed, categories=["CP", "PINAW", "PINAFD"], save_fig=True)
    else:
        print(f"- Skip PI Parallel Coordinates Plot!")
    
def run_one_experiment(seed, override_checkpoints=False, testing=False):
    data_dict = {
        "t+1": {"train_df": df_train_1, "valid_df": df_valid_1, "test_df": df_test_1, "outputs": pred_cols_1},
        "t+2": {"train_df": df_train_2, "valid_df": df_valid_2, "test_df": df_test_2, "outputs": pred_cols_2},
        "t+3": {"train_df": df_train_3, "valid_df": df_valid_3, "test_df": df_test_3, "outputs": pred_cols_3},
    }
    checkpoint_dict = {
        "Train RUE and Predictor": train_rue_n_predictor,
        "Prediction with RUE and Predictor": predict_rue_n_predictor,
        "Prediction with Infer-Noise": predict_infernoise,
        "Train GPR": train_gpr,
        "Prediction with GPR": predict_gpr,
        "Train BNN": train_bnn, 
        "Prediction with BNN": predict_bnn, 
        "Train DER": train_der, # <-
        "Prediction with DER": predict_der,
        "Create DF Dict": create_df_dict,
        "Generate KNN PI":generate_knn_pi,
        "Generate Weighted PI": generate_weighted_pi,
        "Generate Conditional PI": generate_conditional_pi,
        "Generate Copula PI": generate_copula_pi,
        "Generate Conformal PI": generate_conformal_pi,
        "Evaluate Model Performance": evaluate_model_perf,
        "Evaluate UE Performance": evaluate_ue_perf,
        "Evaluate PI Performance": evaluate_pi_perf
    }
    create_all_seed_folders(seed)
    for i, (checkpoint_name, checkpoint_func) in enumerate(checkpoint_dict.items()):
        print(f"{i+1}. {checkpoint_name}")
        start = time.time()
        checkpoint_func(seed, data_dict, override_checkpoints, testing)
        print(f"- {checkpoint_name} took {time.time()-start} s.")

def run_multiple_experiments(seed_list, override_checkpoints=False, testing=False):
    with tqdm(seed_list, total=len(seed_list)) as pbar:
        for seed in pbar:
            pbar.set_description(str(seed))
            run_one_experiment(seed, override_checkpoints, testing)

start_seed = 2024
num_experiments = 4
seed_list = range(start_seed, start_seed+num_experiments)
run_multiple_experiments(seed_list, override_checkpoints=False, testing=False)

### Analyse Results

In [ ]:
seed_list=list(range(2023, 2023+5))
def combine_mean_n_std_matrices(mean, std):
    assert mean.shape == std.shape
    shape = mean.shape
    returned_list = []
    for i in range(shape[0]):
        cur_list = []
        for j in range(shape[1]):
            cur_list.append(f"{mean[i][j]:.3f} ± {std[i][j]:.3f}")
        returned_list.append(cur_list)
    return returned_list

def get_average_of_all_seed_csvs(seed_list, fp_folder, filename, index_col=None):
    result_list = []
    for cur_seed in seed_list:
        fp_perf = os.path.join(fp_folder, str(cur_seed), filename)
        df = pd.read_csv(fp_perf, index_col=0)
        if index_col:
            df = df.set_index(index_col)
        result_list.append(df.values)
    results = np.array(result_list)
    combined_mean = np.mean(results, axis=0)
    combined_std = np.std(results, axis=0)
    return pd.DataFrame(
        combine_mean_n_std_matrices(combined_mean, combined_std), 
        index=df.index, columns=df.columns)

def get_mean_of_all_seed_csvs(seed_list, fp_folder, filename, index_col=None):
    result_list = []
    for cur_seed in seed_list:
        fp_perf = os.path.join(fp_folder, str(cur_seed), filename)
        df = pd.read_csv(fp_perf, index_col=0)
        if index_col:
            df = df.set_index(index_col)
        result_list.append(df.values)
    results = np.array(result_list)
    combined_mean = np.mean(results, axis=0)
    return pd.DataFrame(
        combined_mean, 
        index=df.index, columns=df.columns)

best_color = "#88E7B8"
second_best_color = "#FAC05E"

# Function to highlight most and second highest
def highlight_first_n_second_highest(s):
    s = s.map(lambda x: float(x.split(" ")[0]))
    highest = s.nlargest(1).iloc[-1] # Find the highest value
    second_highest = s.nlargest(2).iloc[-1]  # Find the second highest value
    output = []
    for v in s:
        if v == highest:
            output.append(f'background-color: {best_color}')
        elif v == second_highest:
            output.append(f'background-color: {second_best_color}')
        else:
            output.append("")
    return output

def highlight_first_n_second_lowest(s):
    s = s.map(lambda x: float(x.split(" ")[0]))
    smallest = s.nsmallest(1).iloc[-1] # Find the highest value
    second_smallest = s.nsmallest(2).iloc[-1]  # Find the second highest value
    output = []
    for v in s:
        if v == smallest:
            output.append(f'background-color: {best_color}')
        elif v == second_smallest:
            output.append(f'background-color: {second_best_color}')
        else:
            output.append("")
    return output

#### Prediction Performance

In [ ]:
def get_consolidated_pred_perf(seed_list):
    return get_average_of_all_seed_csvs(seed_list, fp_project_model_evaluations, filename="pred_perf.csv", index_col="Model")

def display_consolidated_pred_perf(pred_perf_df):
    display(pred_perf_df.style.apply(highlight_first_n_second_lowest))

pred_perf_df = get_consolidated_pred_perf(seed_list)
# display(pred_perf_df)
display_consolidated_pred_perf(pred_perf_df)
pred_perf_df.to_csv(os.path.join(fp_project_consolidated_results, "pred_perf.csv"))

In [ ]:
# Function to bold the best
def bold_best(s, direction):
    s = s.map(lambda x: float(x.replace(r"\underline{", "").replace("}", "").split(" ")[0]))
    if direction == "max":
        best = s.nlargest(1).iloc[-1] # Find the best
    elif direction == "min":
        best = s.nsmallest(1).iloc[-1] # Find the best
    else:
        raise Exception(f"Invalid direction {direction}!")
    
    output = []
    for v in s:
        if v == best:
            output.append("textbf:--rwrap;")
        else:
            output.append("")
    return output

# Function to underline the second best
def underline_second_best(s, direction):
    ori_s = s.copy()
    s = s.map(lambda x: float(x.split(" ")[0]))
    if direction == "max":
        second_best = s.nlargest(2).iloc[-1]  # Find the second best
    elif direction == "min":
        second_best = s.nsmallest(2).iloc[-1]  # Find the second best
    else:
        raise Exception(f"Invalid direction {direction}!")
    
    output = []
    for ori_v, v in zip(ori_s, s):
        if v == second_best:
            output.append(r'\underline{'+ori_v+'}')
        else:
            output.append(ori_v)
    return output

def df_to_latex(df, column_format_dict):
    df = df.copy()
    for col, direction in column_format_dict.items():
        df[col] = underline_second_best(df[col], direction)
    styler = df.style
    # Bold column names
    styler.map_index(lambda v: "textbf:--rwrap;", axis="columns")
    # Bold best
    for col, direction in column_format_dict.items():
        styler.apply(bold_best, subset=[col], direction=direction)
    return styler.to_latex(column_format='c'*(df.shape[1]+df.index.nlevels))

print(df_to_latex(pred_perf_df, column_format_dict={"t+1": "min", "t+2": "min", "t+3": "min"}))

#### UE Performance

In [ ]:
def get_consolidated_ue_perf(seed_list):
    return get_average_of_all_seed_csvs(seed_list, fp_project_model_evaluations, filename="ue_perf.csv", index_col="Model")

ue_perf_df = get_consolidated_ue_perf(seed_list)
display(ue_perf_df)
ue_perf_df.to_csv(os.path.join(fp_project_consolidated_results, "ue_perf.csv"))

In [ ]:
def display_consolidated_ue_perf(ue_perf_df):
    ue_perf_df = ue_perf_df.copy()
    # Split df into time label
    num_time, num_metrics = 3, 7
    for i in range(num_time):
        print(f"t+{i}:")
        column_indices = list(range(i*num_metrics, (i+1)*num_metrics))
        cur_df = ue_perf_df.iloc[:,column_indices]
        cur_df.columns = cur_df.columns.str.split(" ").str[-1] # remove time label from column names
        display(
            cur_df.style.apply(
                highlight_first_n_second_highest, subset=cur_df.columns[0]).apply(
                    highlight_first_n_second_lowest, subset=cur_df.columns[2:]
                )
        )
        
display_consolidated_ue_perf(ue_perf_df)

In [ ]:
# Function to bold the best
def bold_best_grouped(s, direction):
    s = s.map(lambda x: float(x.replace(r"\underline{", "").replace("}", "").split(" ")[0]))
    if direction == "max":
        best = s.nlargest(1).iloc[-1] # Find the best
    elif direction == "min":
        best = s.nsmallest(1).iloc[-1] # Find the best
    else:
        raise Exception(f"Invalid direction {direction}!")
    
    output = []
    for v in s:
        if v == best:
            output.append("textbf:--rwrap;")
        else:
            output.append("")
    return output

# Function to underline the second best
def bold_best_underline_second_best_grouped(s, direction, group_col):
    s = s.copy()
    output = []
    for group_name, group_s in s.groupby(level=group_col):
        ori_group_s = group_s.copy()
        group_s = group_s.map(lambda x: float(x.split(" ")[0]))
        if direction == "max":
            best = group_s.nlargest(1).iloc[-1] # Find the best
            second_best = group_s.nlargest(2).iloc[-1]  # Find the second best
        elif direction == "min":
            best = group_s.nsmallest(1).iloc[-1] # Find the best
            second_best = group_s.nsmallest(2).iloc[-1]  # Find the second best
        else:
            raise Exception(f"Invalid direction {direction}!")
        for ori_v, v in zip(ori_group_s, group_s):
            if v == best:
                output.append(r'\textbf{'+ori_v+'}')
            elif v == second_best:
                output.append(r'\underline{'+ori_v+'}')
            else:
                output.append(ori_v)
    return output

def ue_perf_to_latex(ue_perf_df):
    # TODO: Groupby + df to latex
    ue_perf_df = ue_perf_df.copy()
    
    # Split df into time label
    num_time, num_metrics = 3, 7
    all_dfs = []
    for i in range(num_time):
        print()
        column_indices = list(range(i*num_metrics, (i+1)*num_metrics))
        cur_df = ue_perf_df.iloc[:,column_indices]
        cur_df.columns = cur_df.columns.str.split(" ").str[-1] # remove time label from column names
        cur_df.loc[:,"Time Horizon"] = f"t+{i}"
        all_dfs.append(cur_df)
    all_dfs = pd.concat(all_dfs)
    all_dfs = all_dfs.reset_index().set_index(["Time Horizon", "Model"])
    column_format_dict={"Corr": "max", "AURC":"min", "Sigma=0.1": "min", "Sigma=0.2": "min", "Sigma=0.3": "min", "Sigma=0.4": "min"}
    for col, direction in column_format_dict.items():
        all_dfs[col] = bold_best_underline_second_best_grouped(all_dfs[col], direction , group_col="Time Horizon")
    
    return all_dfs.to_latex(column_format='c'*(all_dfs.shape[1]+all_dfs.index.nlevels))
print(ue_perf_to_latex(ue_perf_df))

#### PI Performance

In [ ]:
def get_consolidated_pi_perf(seed_list):
    return get_average_of_all_seed_csvs(seed_list, fp_project_model_evaluations, filename="pi_perf.csv", index_col=["Time Horizon","Method"])

pi_perf_df = get_consolidated_pi_perf(seed_list)
display(pi_perf_df)
pi_perf_df.to_csv(os.path.join(fp_project_consolidated_results, "pi_perf.csv"))

In [ ]:
def plot_pi_mean_metrics_parallel_coordinate(seed_list, categories=["CP", "PINAW", "PINAFD"], save_fig=False):
    from matplotlib import cm
    from matplotlib.colors import rgb2hex
    import seaborn as sns

    pi_stats = get_mean_of_all_seed_csvs(seed_list, fp_project_model_evaluations, filename="pi_perf.csv", index_col=["Time Horizon","Method"]).reset_index()
    
    pi_stats = pi_stats.copy().sort_values(by=["Time Horizon", "Method"])
    num_cat = len(categories)
    
    rue_colors = sns.color_palette('spring', n_colors=4)
    other_colors = sns.color_palette('winter', n_colors=3)
    color_list = sns.color_palette('rainbow', n_colors=pi_stats["Method"].nunique())

    fig, axes = plt.subplots(1, num_cat, sharey=True, dpi=300, figsize=(num_cat*3, 3))
    xticks = range(num_cat)
    
    for i, (time_horizon_label, time_df) in enumerate(pi_stats.groupby("Time Horizon")):
        time_df[categories] = (time_df[categories]-time_df[categories].min())/(time_df[categories].max()-time_df[categories].min())
        for j, (method_label, method_df) in enumerate(time_df.groupby("Method")):
            # print(method_df[categories].values.flatten())
            axes[i].plot(xticks, method_df[categories].values.flatten(), label=method_label if i==0 else None, linestyle="--" if "RUE" not in method_label else "-")
            axes[i].set_xticks(xticks, categories)
        axes[i].set_title(time_horizon_label)

    reorder=lambda hl,nc:(sum((lis[i::nc]for i in range(nc)),[])for lis in hl)
    h_l = axes[0].get_legend_handles_labels()
    fig.legend(*reorder(h_l, 5), loc='upper center', bbox_to_anchor=(0.5, 0), ncol=5, fontsize=8)
    plt.tight_layout()
    if save_fig:
        plt.savefig(os.path.join(fp_project_consolidated_results, "parallel_coord.jpg"), bbox_inches="tight")

In [ ]:
plot_pi_mean_metrics_parallel_coordinate(seed_list, save_fig=True)

#### Runtime

In [ ]:
def analyse_runtime():
    categories = ["Training", "Predicting", "PI"]
    time_df = pd.read_csv(fp_time_log, header=None)
    time_df.columns = ["Seed", "Description", "Time/s"]
    time_df = time_df.drop_duplicates(keep="last")
    time_series = time_df.groupby(by=["Description"])["Time/s"].mean()
    category_values = [category for op in time_series.index for category in categories if category in op]
    output_df = pd.DataFrame({"Category": category_values, "Operation":time_series.index, "Time/s": time_series.values})
    return output_df.sort_values(by=["Category", "Time/s"]).set_index(["Category", "Operation"])

runtime_df = analyse_runtime()
display(runtime_df)
runtime_df.to_csv(os.path.join(fp_project_consolidated_results, "runtime.csv"))